In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 전처리

In [ ]:
!pip -q install pyarrow pandas fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

FILE_PATH = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output/discharge_test_output.parquet"

pf = pq.ParquetFile(FILE_PATH)

print(pf.schema_arrow)

N_PREVIEW = 1000  # 필요 시 조정

# 첫 Row Group만 읽고 그 중 N_PREVIEW 만큼만 슬라이스
tbl_first_rg = pf.read_row_groups([0])  # 첫 RowGroup
tbl_preview  = tbl_first_rg.slice(0, min(N_PREVIEW, tbl_first_rg.num_rows))
df_preview   = tbl_preview.to_pandas(types_mapper=pd.ArrowDtype)

print(f"미리보기 로드 완료: {len(df_preview)} rows")
df_preview.head(10)

subject_id: int64
hadm_id: int64
note_id: string
bhc_text: string
di_text: string
has_bhc: bool
has_di: bool
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 869
미리보기 로드 완료: 1000 rows


,subject_id,hadm_id,note_id,bhc_text,di_text,has_bhc,has_di
0,10000883,25221576,10000883-DS-4,1) Psychiatric: Pt arrived on floor denying pa...,Please take medications as prescribed. Please ...,True,True
1,10001176,23334588,10001176-DS-17,,"Dear Ms. ___, You were admitted to the hospi...",False,True
2,10001667,22672901,10001667-DS-10,Ms. ___ is a ___ year old female with AFib on ...,"Dear Ms. ___, You were hospitalized due to sy...",True,True
3,10002131,27411540,10002131-DS-15,,You were admitted with an acute diarrheal illn...,False,True
4,10002131,24065018,10002131-DS-16,"___ is a ___ y/o F with PMHx CHF, Afib not on ...","Dear ___, It was a pleasure taking care of y...",True,True
5,10002430,27218502,10002430-DS-4,Mr. ___ was admitted to ___ ___ on ___ after ...,"Dear Mr. ___, It was a pleasure taking care of...",True,True
6,10002430,24513842,10002430-DS-5,Mr. ___ is an ___ year old gentleman with hist...,"Dear Mr. ___, You were hospitalized for progr...",True,True
7,10002430,24648311,10002430-DS-6,TRANSITIONAL ISSUES ==================== DIS...,========================= ===================...,True,True
8,10002430,26295318,10002430-DS-7,TRANSITIONAL ISSUES: ==================== [] C...,"Dear Mr. ___, It was a pleasure taking care o...",True,True
9,10002443,21329021,10002443-DS-15,SUMMARY: ===================== ___ male with...,"Dear Mr. ___, WHY WERE YOU ADMITTED TO THE...",True,True


In [ ]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

FILE_PATH = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output/discharge_test_output.parquet"
OUT_JSONL = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/bhc_di_top10k.jsonl"
OUT_PARQUET = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/bhc_di_top10k.parquet"

os.makedirs(os.path.dirname(OUT_JSONL), exist_ok=True)

# 1) RowGroup 단위로 상위 N행만 수집
pf = pq.ParquetFile(FILE_PATH)
want = 30_000
tables, taken = [], 0

for rg in range(pf.num_row_groups):
    if taken >= want:
        break
    t = pf.read_row_groups([rg])  # 이 RowGroup 전체
    remain = want - taken
    if t.num_rows > remain:
        t = t.slice(0, remain)
    tables.append(t)
    taken += t.num_rows

tbl = pa.concat_tables(tables) if len(tables) > 1 else tables[0]
df = tbl.to_pandas(types_mapper=pd.ArrowDtype)

# 2) di_text만 남기기(가장 안전)
keep_cols = ["di_text"]
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise KeyError(f"필수 컬럼이 없습니다: {missing}")

df = df[keep_cols].copy()
df["di_text"] = df["di_text"].astype("string").fillna("")

# 3) 저장
df.to_json(OUT_JSONL, orient="records", lines=True, force_ascii=False)
df.to_parquet(OUT_PARQUET, engine="pyarrow", index=False)

print("완료!")
print("행 수:", len(df))
print("JSONL:", OUT_JSONL)
print("PARQUET:", OUT_PARQUET)
print("\n샘플 2행:")
print(df.head(2))


완료!
행 수: 30000
JSONL: /content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/bhc_di_top10k.jsonl
PARQUET: /content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/bhc_di_top10k.parquet

샘플 2행:
                                             di_text
0  Please take medications as prescribed.
Please ...
1  Dear Ms. ___, 

You were admitted to the hospi...


# 벡터 DB 구축

In [ ]:
!pip -q install pandas pyarrow sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.6 MB/s eta 0:00:00


In [ ]:
# ==== JSONL(di_text) -> LangChain FAISS 인덱스 생성 ====
# 결과: RAG_FAISS_DIR 폴더 안에 index.faiss + index.pkl 생성

import os
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 경로/설정
DATA_PATH     = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/bhc_di_top10k.jsonl"  # di_text만 있는 JSONL
TEXT_COL      = "di_text"
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"             # 저장 폴더(기존과 동일)
E5_MODEL      = "intfloat/e5-base-v2"   # 쿼리에서 쓰는 모델과 반드시 동일

os.makedirs(RAG_FAISS_DIR, exist_ok=True)

# 1) 데이터 로드
df = pd.read_json(DATA_PATH, lines=True)
assert TEXT_COL in df.columns, f"필수 컬럼({TEXT_COL})이 없습니다."

texts = (
    df[TEXT_COL].astype(str)
      .fillna("")
      .str.strip()
)
texts = texts[texts != ""].drop_duplicates().tolist()
print("문서 수(중복제거):", len(texts))

# 2) E5 임베딩 준비 (코사인 정규화)
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)

# 3) E5 권장 프리픽스 적용 후 인덱스 생성 & 저장
texts_with_prefix = [("passage: " + t) for t in texts]
vs = FAISS.from_texts(texts_with_prefix, embedding=emb)  # 내부에서 임베딩→FAISS 구성
vs.save_local(RAG_FAISS_DIR)  # index.faiss + index.pkl 생성

print("✅ LangChain 포맷 저장 완료:", RAG_FAISS_DIR)
print("생성 파일:", sorted(os.listdir(RAG_FAISS_DIR)))


문서 수(중복제거): 28049
✅ LangChain 포맷 저장 완료: /content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index
생성 파일: ['index.faiss', 'index.pkl']


# RAG 데이터셋 이용

In [ ]:
# 충돌 가능 패키지 정리
!pip -q uninstall -y bitsandbytes pandas pyarrow rich requests || true

# Colab 기본과 맞추고, bnb/transformers는 최신으로
!pip -q install \
  "pandas==2.2.2" \
  "pyarrow>=14,<20" \
  "requests==2.32.4" \
  "rich<14" \
  "bitsandbytes>=0.43.3" \
  "transformers>=4.44.2" \
  "accelerate>=0.33.0" \
  "peft>=0.12.0" \
  "trl" \
  "sentence-transformers>=2.5.1" \
  "faiss-cpu==1.8.0.post1" \
  "langchain==0.2.14" "langchain-community==0.2.12"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U transformers accelerate peft datasets pandas bitsandbytes rich
# !pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) DI 전용 FAISS/E5 경로 ====
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH
print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# 입력/출력 결합 (동일 케이스 기준)
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")

# 입력 텍스트 컬럼명 가정: "text"
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

# 완전 빈 케이스 제거
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# 소량 샘플링(데모)
if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# (중요) DI 프롬프트를 STRICT/ SOFT 두 벌로 분리
DI_INSTR_STRICT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

INPUT NOTE RULES
- Use ONLY explicit instructions present in the INPUT NOTE (medication directions/changes with dose/frequency if present; activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.
"""

DI_INSTR_SOFT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

ALLOWED CONTENT (when INPUT has no explicit DI)
- You MAY provide general, standard, patient-facing guidance appropriate for the described condition/procedure:
  activity limits, wound/incision care, when-to-call/ER triggers, generic follow-up wording, diet/hydration.
HARD RESTRICTIONS
- DO NOT invent any new medication names, doses, frequencies, or specific appointment dates/locations not present in INPUT.
- Avoid imaging/lab/result narrative (CT/MRI/pathology).
STYLE
- Write 6–8 concise bullets, each starting with "- " and a directive verb (e.g., Take/Continue/Avoid/Call/Walk/Keep).
"""

RAG_STRICT_BLOCK = """
RAG USE
- Use retrieved DI exemplars for wording/style only.
- If exemplars conflict with INPUT, prefer INPUT.
"""

RAG_SOFT_BLOCK = """
RAG USE
- Use retrieved DI exemplars to shape style and typical patient-facing phrasing and categories.
- Do NOT copy medication specifics unless they exist in INPUT.
"""

# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# =========================
# 6) 키워드 추출(경량 휴리스틱)
# =========================
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())  # 일반어 추가

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3, 2, 1):
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)}
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen):
            continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms:
            break
    return out

def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]
    return qs if qs else ["query: discharge instructions", "query: patient instructions"]

def retrieve_di_exemplars_by_keywords(note_text: str, retriever, top_per_kw: int = 3, max_total: int = 10):
    kws = extract_disease_keywords_heuristic(note_text)
    queries = build_e5_queries_from_keywords(kws)
    docs, seen = [], set()
    for q in queries:
        hits = retriever.invoke(q)  # langchain deprecation 대응
        for d in hits[:top_per_kw]:
            t = _strip_passage_prefix(d.page_content.strip())
            if not t or len(t) < 60 or len(t) > 3000:
                continue
            if t in seen:
                continue
            seen.add(t)
            docs.append(t)
            if len(docs) >= max_total:
                break
        if len(docs) >= max_total:
            break
    return docs, kws

# =========================
# 7) 프롬프트/생성/후처리 유틸
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=False,
        repetition_penalty=1.20, no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

# --- 실제 DI가 ‘명시적’으로 있는지 판단 (지시문 2개 이상)
_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def _extract_di_chunk(note_text: str) -> str:
    if not note_text: return ""
    m = re.search(r"(?is)(Discharge\s+Instructions?|Patient\s+Instructions?|Discharge\s+Plan)\s*:?\s*(.+?)(\n[A-Z][A-Za-z ]{2,40}\s*:|\Z)", note_text)
    return (m.group(2) if m else "").strip()

def has_explicit_di(note_text: str, min_bullets: int = 3) -> bool:
    chunk = _extract_di_chunk(note_text)
    if not chunk: return False
    cnt = 0
    for ln in chunk.splitlines():
        s = ln.strip()
        if not s: continue
        if (s.startswith(("-", "*")) or _DIRECTIVE_RE.search(s)) and not _NON_INSTR_RE.search(s):
            cnt += 1
    return cnt >= min_bullets

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    """'- '가 없으면 문장 단위로 분할해 강제로 불릿화"""
    if re.search(r"(?m)^\s*-\s+", text):
        return text
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}" for s in sents if not _NON_INSTR_RE.search(s)][:max_bullets]
    return "\n".join(bullets)

def build_di_prompt_with_rag(note_text: str, di_exemplars: list, strict: bool) -> str:
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])
    if strict:
        system = DI_INSTR_STRICT + RAG_STRICT_BLOCK + """
- Output MUST be 5–8 bullet lines, each starting with "- ", and each bullet begins with a directive verb (e.g., "Take", "Continue", "Avoid", "Call").
"""
    else:
        system = DI_INSTR_SOFT + RAG_SOFT_BLOCK
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

# ---- 유사도/일치 검사 & DI 정화 ----
def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 제외
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

def filter_di_to_source(di_text: str, source: str, min_run: int = 12) -> str:
    keep = []
    for b in di_text.splitlines():
        s = b.strip()
        if not s or not s.startswith("- "): continue
        if not _DIRECTIVE_RE.search(s): continue
        if _NON_INSTR_RE.search(s): continue
        if not has_long_overlap(s, source, min_run=min_run): continue
        if _TRIGGER_START.match(s) and not has_long_overlap(s, source, min_run=max(min_run, 20)): continue
        if _DOSE_PAT.search(s) and not _dose_fragments_in_source(s, source): continue
        keep.append(s)
    out, seen = [], set()
    for k in keep:
        if k not in seen: seen.add(k); out.append(k)
    return "\n".join(out)

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 제외
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if has_long_overlap(s_norm, note_text, 10) and s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(키워드→DI 벡터DB RAG)
# =========================
N = min(3, len(sample_df))
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = re.sub(r"\n{3,}", "\n\n", bhc_raw).strip()

    # ---- DI: 키워드 추출 → DI 벡터DB에서 예시 회수 → 프롬프트에 포함 → 생성
    di_exemplars, kws = retrieve_di_exemplars_by_keywords(note_text, retriever, top_per_kw=3, max_total=10)
    print(f"[키워드] {kws if kws else '(none)'} | [회수된 DI 예시] {len(di_exemplars)}개")

    STRICT_MODE = has_explicit_di(note_text)  # '지시문 2개 이상' 기준으로 엄격 여부 판정
    di_prompt   = build_di_prompt_with_rag(note_text, di_exemplars, strict=STRICT_MODE)
    di_raw      = _generate_text(trained_model, di_prompt, max_new_tokens=800)  # 여유

    # ---- DI 후처리: 불릿 강제 → 정규화
    di_norm = normalize_bullets(ensure_bulleted(di_raw))

    if STRICT_MODE:
        # (A) 엄격: 원문 일치성 필터(완화: 10) + 안전 정화 + 카테고리 보강
        di_text = filter_di_to_source(di_norm, note_text, min_run=8)
        di_text = sanitize_di(di_text)
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "No explicit discharge instructions found in source."
    else:
        # (B) 완화: 원문 일치 필터 건너뜀 (약/용량/일정 새창조는 프롬프트로 금지)
        # 지시동사가 하나도 없으면 최소 치환으로 유도
        if not any(_DIRECTIVE_RE.search(ln) for ln in di_norm.splitlines()):
            di_norm = re.sub(r"(?m)^\-\s*(You should|Please\s+|Kindly\s+)?", "- ", di_norm)
        di_text = sanitize_di(di_norm)
        di_text = ensure_core_categories(di_text, note_text)
        # 최후 안전 템플릿
        if not di_text.strip():
            di_text = "\n".join([
                "- Continue your home medications exactly as instructed on your discharge paperwork.",
                "- Walk several times daily and avoid heavy lifting until cleared by your surgeon.",
                "- Keep your incision clean and dry; you may shower but avoid soaking baths or swimming.",
                "- Watch for fever, worsening pain, redness/swelling or drainage from the incision, chest pain, or shortness of breath—seek care or call right away if these occur.",
                "- Schedule and attend follow-up with your surgeon/primary care as instructed.",
                "- Drink fluids and advance diet as tolerated unless told otherwise."
            ])

    # ---- 합치기/출력
    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    # ---- GT(정답)도 함께 출력
    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[키워드] ['a renal cyst', 'mild chronic cholecystitis', 'atypia htn hyperlipidemia', 'htn hyperlipidemia renal', 'renal cyst situation', 'breast atypia htn', 'breast cancer in', 'of breast cancer'] | [회수된 DI 예시] 10개


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
The patient presented on POD+1 from surgery for central 
pancreatectomy and cholecystectomy. She tolerated the procedure 
well without complications. On post op Day 1 she developed some 
nausea which resolved after taking antiemetics. Her NG tube was 
removed on POD +2 and she began eating regular diet. She did not 
develop fever/chills during hospitalization. She remained 
hemodynamically stable throughout stay. She was discharged home 
on POD +3 in stable condition. She will continue to receive 
anti-n

## GPT버전

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U transformers accelerate peft datasets pandas bitsandbytes rich
# !pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) DI 전용 FAISS/E5 경로 ====
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH
print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# 입력/출력 결합 (동일 케이스 기준)
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")

# 입력 텍스트 컬럼명 가정: "text"
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

# 완전 빈 케이스 제거
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# 소량 샘플링(데모)
if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# (중요) DI 프롬프트를 STRICT/ SOFT 두 벌로 분리
DI_INSTR_STRICT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

INPUT NOTE RULES
- Use ONLY explicit instructions present in the INPUT NOTE (medication directions/changes with dose/frequency if present; activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.
"""

DI_INSTR_SOFT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

ALLOWED CONTENT (when INPUT has no explicit DI)
- You MAY provide general, standard, patient-facing guidance appropriate for the described condition/procedure:
  activity limits, wound/incision care, when-to-call/ER triggers, generic follow-up wording, diet/hydration.
HARD RESTRICTIONS
- DO NOT invent any new medication names, doses, frequencies, or specific appointment dates/locations not present in INPUT.
- Avoid imaging/lab/result narrative (CT/MRI/pathology).
STYLE
- Write 6–8 concise bullets, each starting with "- " and a directive verb (e.g., Take/Continue/Avoid/Call/Walk/Keep).
"""

RAG_STRICT_BLOCK = """
RAG USE
- Use retrieved DI exemplars for wording/style only.
- If exemplars conflict with INPUT, prefer INPUT.
"""

RAG_SOFT_BLOCK = """
RAG USE
- Use retrieved DI exemplars to shape style and typical patient-facing phrasing and categories.
- Do NOT copy medication specifics unless they exist in INPUT.
"""

# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# =========================
# 6) 키워드 추출(경량 휴리스틱)
# =========================
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())  # 일반어 추가

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3, 2, 1):
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)}
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen):
            continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms:
            break
    return out

def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]
    return qs if qs else ["query: discharge instructions", "query: patient instructions"]

def retrieve_di_exemplars_by_keywords(note_text: str, retriever, top_per_kw: int = 3, max_total: int = 10):
    kws = extract_disease_keywords_heuristic(note_text)
    queries = build_e5_queries_from_keywords(kws)
    docs, seen = [], set()
    for q in queries:
        hits = retriever.invoke(q)  # langchain deprecation 대응
        for d in hits[:top_per_kw]:
            t = _strip_passage_prefix(d.page_content.strip())
            if not t or len(t) < 60 or len(t) > 3000:
                continue
            if t in seen:
                continue
            seen.add(t)
            docs.append(t)
            if len(docs) >= max_total:
                break
        if len(docs) >= max_total:
            break
    return docs, kws

# =========================
# 7) 프롬프트/생성/후처리 유틸
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=False,
        repetition_penalty=1.20, no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

# --- 실제 DI가 ‘명시적’으로 있는지 판단 (지시문 2개 이상)
_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def _extract_di_chunk(note_text: str) -> str:
    if not note_text: return ""
    m = re.search(r"(?is)(Discharge\s+Instructions?|Patient\s+Instructions?|Discharge\s+Plan)\s*:?\s*(.+?)(\n[A-Z][A-Za-z ]{2,40}\s*:|\Z)", note_text)
    return (m.group(2) if m else "").strip()

def has_explicit_di(note_text: str, min_bullets: int = 2) -> bool:
    chunk = _extract_di_chunk(note_text)
    if not chunk: return False
    cnt = 0
    for ln in chunk.splitlines():
        s = ln.strip()
        if not s: continue
        if (s.startswith(("-", "*")) or _DIRECTIVE_RE.search(s)) and not _NON_INSTR_RE.search(s):
            cnt += 1
    return cnt >= min_bullets

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    """'- '가 없으면 문장 단위로 분할해 강제로 불릿화"""
    if re.search(r"(?m)^\s*-\s+", text):
        return text
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}" for s in sents if not _NON_INSTR_RE.search(s)][:max_bullets]
    return "\n".join(bullets)

def build_di_prompt_with_rag(note_text: str, di_exemplars: list, strict: bool) -> str:
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])
    if strict:
        system = DI_INSTR_STRICT + RAG_STRICT_BLOCK + """
- Output MUST be 5–8 bullet lines, each starting with "- ", and each bullet begins with a directive verb (e.g., "Take", "Continue", "Avoid", "Call").
"""
    else:
        system = DI_INSTR_SOFT + RAG_SOFT_BLOCK
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

# ---- 유사도/일치 검사 & DI 정화 ----
def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 제외
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

def filter_di_to_source(di_text: str, source: str, min_run: int = 12) -> str:
    keep = []
    for b in di_text.splitlines():
        s = b.strip()
        if not s or not s.startswith("- "): continue
        if not _DIRECTIVE_RE.search(s): continue
        if _NON_INSTR_RE.search(s): continue
        if not has_long_overlap(s, source, min_run=min_run): continue
        if _TRIGGER_START.match(s) and not has_long_overlap(s, source, min_run=max(min_run, 20)): continue
        if _DOSE_PAT.search(s) and not _dose_fragments_in_source(s, source): continue
        keep.append(s)
    out, seen = [], set()
    for k in keep:
        if k not in seen: seen.add(k); out.append(k)
    return "\n".join(out)

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 제외
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if has_long_overlap(s_norm, note_text, 10) and s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(키워드→DI 벡터DB RAG)
# =========================
N = min(3, len(sample_df))
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = re.sub(r"\n{3,}", "\n\n", bhc_raw).strip()

    # ---- DI: 키워드 추출 → DI 벡터DB에서 예시 회수 → 프롬프트에 포함 → 생성
    di_exemplars, kws = retrieve_di_exemplars_by_keywords(note_text, retriever, top_per_kw=3, max_total=10)
    print(f"[키워드] {kws if kws else '(none)'} | [회수된 DI 예시] {len(di_exemplars)}개")

    STRICT_MODE = has_explicit_di(note_text)  # '지시문 2개 이상' 기준으로 엄격 여부 판정
    di_prompt   = build_di_prompt_with_rag(note_text, di_exemplars, strict=STRICT_MODE)
    di_raw      = _generate_text(trained_model, di_prompt, max_new_tokens=800)  # 여유

    # ---- DI 후처리: 불릿 강제 → 정규화
    di_norm = normalize_bullets(ensure_bulleted(di_raw))

    if STRICT_MODE:
        # (A) 엄격: 원문 일치성 필터(완화: 10) + 안전 정화 + 카테고리 보강
        di_text = filter_di_to_source(di_norm, note_text, min_run=10)
        di_text = sanitize_di(di_text)
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "No explicit discharge instructions found in source."
    else:
        # (B) 완화: 원문 일치 필터 건너뜀 (약/용량/일정 새창조는 프롬프트로 금지)
        # 지시동사가 하나도 없으면 최소 치환으로 유도
        if not any(_DIRECTIVE_RE.search(ln) for ln in di_norm.splitlines()):
            di_norm = re.sub(r"(?m)^\-\s*(You should|Please\s+|Kindly\s+)?", "- ", di_norm)
        di_text = sanitize_di(di_norm)
        di_text = ensure_core_categories(di_text, note_text)
        # 최후 안전 템플릿
        if not di_text.strip():
            di_text = "\n".join([
                "- Continue your home medications exactly as instructed on your discharge paperwork.",
                "- Walk several times daily and avoid heavy lifting until cleared by your surgeon.",
                "- Keep your incision clean and dry; you may shower but avoid soaking baths or swimming.",
                "- Watch for fever, worsening pain, redness/swelling or drainage from the incision, chest pain, or shortness of breath—seek care or call right away if these occur.",
                "- Schedule and attend follow-up with your surgeon/primary care as instructed.",
                "- Drink fluids and advance diet as tolerated unless told otherwise."
            ])

    # ---- 합치기/출력
    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    # ---- GT(정답)도 함께 출력
    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[키워드] ['a renal cyst', 'mild chronic cholecystitis', 'atypia htn hyperlipidemia', 'htn hyperlipidemia renal', 'renal cyst situation', 'breast atypia htn', 'breast cancer in', 'of breast cancer'] | [회수된 DI 예시] 10개


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
The patient presented on POD+1 from surgery for central 
pancreatectomy and cholecystectomy. She tolerated the procedure 
well without complications. On post op Day 1 she developed some 
nausea which resolved after taking antiemetics. Her NG tube was 
removed on POD +2 and she began eating regular diet. She did not 
develop fever/chills during hospitalization. She remained 
hemodynamically stable throughout stay. She was discharged home 
on POD +3 in stable condition. She will continue to receive 
anti-n

## 40_000번째부터 시작하는 코드(즉, RAG에 없는 데이터)

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U transformers accelerate peft datasets pandas bitsandbytes rich
# !pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

# (NEW) 순차 슬라이스 모드: 필터 이후 데이터에서 1-based로 START_AT_ROW부터 TEST_LIMIT개 선택
USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000   # 1-based index (필터/병합 후 기준). 40,000번째 행부터 시작

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) DI 전용 FAISS/E5 경로 ====
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH
print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# 입력/출력 결합 (동일 케이스 기준)
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")

# 입력 텍스트 컬럼명 가정: "text"
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

# 완전 빈 케이스 제거
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# ── 샘플 선택 로직 (수정됨) ─────────────────────────────────────────
if USE_SEQUENTIAL_SLICE:
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1  # 0-based로 변환
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    # 기존 무작위 샘플링
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")
# ───────────────────────────────────────────────────────────────────

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# (중요) DI 프롬프트를 STRICT/ SOFT 두 벌로 분리
DI_INSTR_STRICT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

INPUT NOTE RULES
- Use ONLY explicit instructions present in the INPUT NOTE (medication directions/changes with dose/frequency if present; activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.
"""

DI_INSTR_SOFT = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

ALLOWED CONTENT (when INPUT has no explicit DI)
- You MAY provide general, standard, patient-facing guidance appropriate for the described condition/procedure:
  activity limits, wound/incision care, when-to-call/ER triggers, generic follow-up wording, diet/hydration.
HARD RESTRICTIONS
- DO NOT invent any new medication names, doses, frequencies, or specific appointment dates/locations not present in INPUT.
- Avoid imaging/lab/result narrative (CT/MRI/pathology).
STYLE
- Write 6–8 concise bullets, each starting with "- " and a directive verb (e.g., Take/Continue/Avoid/Call/Walk/Keep).
"""

RAG_STRICT_BLOCK = """
RAG USE
- Use retrieved DI exemplars for wording/style only.
- If exemplars conflict with INPUT, prefer INPUT.
"""

RAG_SOFT_BLOCK = """
RAG USE
- Use retrieved DI exemplars to shape style and typical patient-facing phrasing and categories.
- Do NOT copy medication specifics unless they exist in INPUT.
"""

# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# =========================
# 6) 키워드 추출(경량 휴리스틱)
# =========================
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3, 2, 1):
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)}
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen):
            continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms:
            break
    return out

def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]
    return qs if qs else ["query: discharge instructions", "query: patient instructions"]

def retrieve_di_exemplars_by_keywords(note_text: str, retriever, top_per_kw: int = 3, max_total: int = 10):
    kws = extract_disease_keywords_heuristic(note_text)
    queries = build_e5_queries_from_keywords(kws)
    docs, seen = [], set()
    for q in queries:
        hits = retriever.invoke(q)
        for d in hits[:top_per_kw]:
            t = _strip_passage_prefix(d.page_content.strip())
            if not t or len(t) < 60 or len(t) > 3000:
                continue
            if t in seen:
                continue
            seen.add(t)
            docs.append(t)
            if len(docs) >= max_total:
                break
        if len(docs) >= max_total:
            break
    return docs, kws

# =========================
# 7) 프롬프트/생성/후처리 유틸
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=False,
        repetition_penalty=1.20, no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def _extract_di_chunk(note_text: str) -> str:
    if not note_text: return ""
    m = re.search(r"(?is)(Discharge\s+Instructions?|Patient\s+Instructions?|Discharge\s+Plan)\s*:?\s*(.+?)(\n[A-Z][A-Za-z ]{2,40}\s*:|\Z)", note_text)
    return (m.group(2) if m else "").strip()

def has_explicit_di(note_text: str, min_bullets: int = 3) -> bool:
    chunk = _extract_di_chunk(note_text)
    if not chunk: return False
    cnt = 0
    for ln in chunk.splitlines():
        s = ln.strip()
        if not s: continue
        if (s.startswith(("-", "*")) or _DIRECTIVE_RE.search(s)) and not _NON_INSTR_RE.search(s):
            cnt += 1
    return cnt >= min_bullets

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    if re.search(r"(?m)^\s*-\s+", text):
        return text
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}" for s in sents if not _NON_INSTR_RE.search(s)][:max_bullets]
    return "\n".join(bullets)

def build_di_prompt_with_rag(note_text: str, di_exemplars: list, strict: bool) -> str:
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])
    if strict:
        system = DI_INSTR_STRICT + RAG_STRICT_BLOCK + """
- Output MUST be 5–8 bullet lines, each starting with "- ", and each bullet begins with a directive verb (e.g., "Take", "Continue", "Avoid", "Call").
"""
    else:
        system = DI_INSTR_SOFT + RAG_SOFT_BLOCK
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

def filter_di_to_source(di_text: str, source: str, min_run: int = 12) -> str:
    keep = []
    for b in di_text.splitlines():
        s = b.strip()
        if not s or not s.startswith("- "): continue
        if not _DIRECTIVE_RE.search(s): continue
        if _NON_INSTR_RE.search(s): continue
        if not has_long_overlap(s, source, min_run=min_run): continue
        if _TRIGGER_START.match(s) and not has_long_overlap(s, source, min_run=max(min_run, 20)): continue
        if _DOSE_PAT.search(s) and not _dose_fragments_in_source(s, source): continue
        keep.append(s)
    out, seen = [], set()
    for k in keep:
        if k not in seen: seen.add(k); out.append(k)
    return "\n".join(out)

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if has_long_overlap(s_norm, note_text, 10) and s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(키워드→DI 벡터DB RAG)
# =========================
N = len(sample_df)  # (수정) 선택된 표본 전체 처리
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = re.sub(r"\n{3,}", "\n\n", bhc_raw).strip()

    # ---- DI: 키워드 추출 → DI 벡터DB에서 예시 회수 → 프롬프트에 포함 → 생성
    di_exemplars, kws = retrieve_di_exemplars_by_keywords(note_text, retriever, top_per_kw=3, max_total=10)
    print(f"[키워드] {kws if kws else '(none)'} | [회수된 DI 예시] {len(di_exemplars)}개")

    STRICT_MODE = has_explicit_di(note_text)
    di_prompt   = build_di_prompt_with_rag(note_text, di_exemplars, strict=STRICT_MODE)
    di_raw      = _generate_text(trained_model, di_prompt, max_new_tokens=800)

    # ---- DI 후처리
    di_norm = normalize_bullets(ensure_bulleted(di_raw))

    if STRICT_MODE:
        di_text = filter_di_to_source(di_norm, note_text, min_run=8)
        di_text = sanitize_di(di_text)
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "No explicit discharge instructions found in source."
    else:
        if not any(_DIRECTIVE_RE.search(ln) for ln in di_norm.splitlines()):
            di_norm = re.sub(r"(?m)^\-\s*(You should|Please\s+|Kindly\s+)?", "- ", di_norm)
        di_text = sanitize_di(di_norm)
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "\n".join([
                "- Continue your home medications exactly as instructed on your discharge paperwork.",
                "- Walk several times daily and avoid heavy lifting until cleared by your surgeon.",
                "- Keep your incision clean and dry; you may shower but avoid soaking baths or swimming.",
                "- Watch for fever, worsening pain, redness/swelling or drainage from the incision, chest pain, or shortness of breath—seek care or call right away if these occur.",
                "- Schedule and attend follow-up with your surgeon/primary care as instructed.",
                "- Drink fluids and advance diet as tolerated unless told otherwise."
            ])

    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 선택 모드: 순차 슬라이스 | 시작(1-based): 40,000 | 선택 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[키워드] ['hepatic', 'biliary', 'partially circumferential hyperenhancing', 'new partially circumferential', 'catheter within appropriate', 'retrocardiac opacity with', 'decrease in retrocardiac', 'in retrocardiac opacity'] | [회수된 DI 예시] 10개


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
Ms. ___ is a ___ y/o woman who presented to ___ after being 
admitted to ___ for failure to thrive secondary to inability to 
tolerate enteral feeds following placement of a PTBD. Her diet 
had been increased gradually since hospitalization at ___ but 
she continued to have episodes of nausea/vomiting and diarrhea. 
She also developed a fever and leukocytosis prior to transfer to 
___. Upon arrival to our service, she underwent a CT abdomen 
which rev

# Qwen3에게 용어 추출해서 검색하는 방식

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U transformers accelerate peft datasets pandas bitsandbytes rich
# !pip -q install langchain langchain-community faiss-cpu sentence-transformers
!pip -q install rank-bm25 sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from collections import defaultdict, Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

# (NEW) 순차 슬라이스 모드: 필터 이후 데이터에서 1-based로 START_AT_ROW부터 TEST_LIMIT개 선택
USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000   # 1-based index (필터/병합 후 기준)

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) DI 전용 FAISS/E5 경로 ====
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH
print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# 입력/출력 결합 (동일 케이스 기준)
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")

# 입력 텍스트 컬럼명 가정: "text"
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

# 완전 빈 케이스 제거
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# ── 샘플 선택 로직 ─────────────────────────────────────────
if USE_SEQUENTIAL_SLICE:
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1  # 0-based로 변환
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    # 기존 무작위 샘플링
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")
# ─────────────────────────────────────────────────────────

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the "Brief Hospital Course (BHC)"
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.

STRICT FORMAT
- Output MUST be a single compact paragraph (≤8 sentences).
- Do NOT use bullets, enumerations, problem-list headers, or any line starting with '#'.
- Do NOT introduce section headers (e.g., "TRANSITIONAL ISSUES", "ADMISSION DIAGNOSES", "Chronic Conditions", etc.).

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# (중요) DI 프롬프트를 STRICT/ SOFT 두 벌로 분리
DI_INSTR_STRICT = """
You are an expert physician-writer. Produce "Discharge Instructions (DI)".

INPUT NOTE RULES
- Use ONLY explicit instructions present in the INPUT NOTE (medication directions/changes with dose/frequency if present; activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.
"""

DI_INSTR_SOFT = """
You are an expert physician-writer. Produce "Discharge Instructions (DI)".

ALLOWED CONTENT (when INPUT has no explicit DI)
- You MAY provide general, standard, patient-facing guidance appropriate for the described condition/procedure:
  activity limits, wound/incision care, when-to-call/ER triggers, generic follow-up wording, diet/hydration.
HARD RESTRICTIONS
- DO NOT invent any new medication names, doses, frequencies, numeric thresholds (weights/temperatures/volumes), or specific appointment dates/locations not present in INPUT.
- Avoid imaging/lab/result narrative (CT/MRI/pathology).
STYLE
- Write 6–8 concise bullets, each starting with "- " and a directive verb (e.g., Take/Continue/Avoid/Call/Walk/Keep).
"""

RAG_STRICT_BLOCK = """
RAG USE
- Use retrieved DI exemplars for wording/style only.
- If exemplars conflict with INPUT, prefer INPUT.
"""

RAG_SOFT_BLOCK = """
RAG USE
- Use retrieved DI exemplars to shape style and typical patient-facing phrasing and categories.
- Do NOT copy medication specifics or numeric thresholds unless they exist in INPUT.
"""

# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# =========================
# 6) 키워드 추출(경량 휴리스틱)
# =========================
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3, 2, 1):
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)}
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen):
            continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms:
            break
    return out

def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]
    return qs if qs else ["query: discharge instructions", "query: patient instructions"]

# =========================
# 6-1) (NEW) 하이브리드 RAG: LLM 질의 + BM25 + FAISS + RRF (+옵션 리랭커)
# =========================
USE_LLM_QUERY      = True
LLM_QUERY_K        = 6
USE_HYBRID         = True
FAISS_CAND_K       = 50
BM25_CAND_K        = 80
FUSE_TOPK          = 12
USE_RERANKER       = False
RERANKER_MODEL     = "cross-encoder/ms-marco-MiniLM-L-6-v2"

_BM25 = None
def _load_di_mapping(prefix: str):
    mapping_path = prefix + ".mapping.jsonl"
    if not os.path.exists(mapping_path):
        alt = os.path.join(os.path.dirname(prefix), "di_index.mapping.jsonl")
        if os.path.exists(alt):
            mapping_path = alt
        else:
            raise FileNotFoundError(f"DI mapping jsonl not found: {mapping_path}")
    texts, ids = [], []
    with open(mapping_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            txt = (obj.get("di_text","") or "").strip()
            if not txt:
                continue
            texts.append(txt)
            ids.append(obj.get("doc_id"))
    docs = [Document(page_content=t, metadata={"doc_id":i}) for t,i in zip(texts, ids)]
    return docs

def get_bm25_retriever(prefix: str):
    global _BM25
    if _BM25 is not None:
        return _BM25
    docs = _load_di_mapping(prefix)
    bullets = []
    for d in docs:
        for ln in d.page_content.splitlines():
            s = ln.strip()
            if len(s) < 20:
                continue
            if re.search(r"^(Discharge|Service|Date|Attending)\b", s, re.I):
                continue
            bullets.append(Document(page_content=s, metadata=d.metadata))
    _BM25 = BM25Retriever.from_documents(bullets if bullets else docs)
    _BM25.k = BM25_CAND_K
    return _BM25

def rrf_fuse(ranklists, k=FUSE_TOPK, c=60):
    agg = defaultdict(float)
    seen_doc = {}
    for rl in ranklists:
        for rank, (doc, _) in enumerate(rl, start=1):
            key = (doc.page_content[:200], doc.metadata.get("doc_id"))
            seen_doc[key] = doc
            agg[key] += 1.0 / (c + rank)
    fused = sorted(agg.items(), key=lambda x: x[1], reverse=True)
    fused_docs = [seen_doc[k] for k,_ in fused[:k]]
    return fused_docs

_reranker = None
def _get_reranker():
    global _reranker
    if _reranker is not None:
        return _reranker
    try:
        from sentence_transformers import CrossEncoder
        _reranker = CrossEncoder(RERANKER_MODEL)
    except Exception as e:
        print(f"[warn] reranker load failed: {e}")
        _reranker = None
    return _reranker

def rerank_with_cross_encoder(query: str, docs: list):
    ce = _get_reranker()
    if ce is None:
        return docs
    pairs = [(query, d.page_content) for d in docs]
    scores = ce.predict(pairs)
    order = np.argsort(scores)[::-1]
    return [docs[i] for i in order]

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=False,
        repetition_penalty=1.20, no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def llm_make_queries(note_text: str, k: int = LLM_QUERY_K) -> list:
    if not USE_LLM_QUERY or not note_text.strip():
        return []
    prompt = (
        "Extract up to 8 short search queries for retrieving patient-facing discharge instruction examples.\n"
        "Cover these categories if present: DIAGNOSIS, PROCEDURE, DEVICE/DRAIN (e.g., PTBD), COMPLICATION, FOLLOW-UP.\n"
        "Return as a JSON array of strings. Keep each query ≤6 words. Avoid generic words.\n\n"
        f"INPUT NOTE:\n{note_text.strip()}\n"
    )
    try:
        raw = _generate_text(trained_model, prompt, max_new_tokens=200)
        m = re.search(r"\[[\s\S]*\]", raw)
        arr = json.loads(m.group(0)) if m else json.loads(raw)
        expanded = []
        syn = {
            "ptbd": ["percutaneous transhepatic biliary drain", "biliary drain", "biliary catheter"],
            "jp drain": ["surgical drain", "jackson-pratt drain"],
            "cholecystectomy": ["post-cholecystectomy", "gallbladder surgery"],
            "pancreatectomy": ["post-pancreatectomy"],
        }
        for q in arr:
            q2 = str(q).strip().lower()
            if not q2 or len(q2) < 3:
                continue
            expanded.append(q2)
            for k,v in syn.items():
                if k in q2:
                    expanded.extend(v)
        out = []
        for s in expanded:
            if s not in out:
                out.append(s)
        return out[:k] if k else out
    except Exception as e:
        print(f"[warn] LLM query gen failed: {e}")
        return []

def retrieve_di_exemplars_hybrid(note_text: str, faiss_vs, prefix: str, top_per_q: int = 12, max_total: int = 12):
    kws    = extract_disease_keywords_heuristic(note_text, max_terms=8)
    llm_qs = llm_make_queries(note_text, k=LLM_QUERY_K)
    base_qs = build_e5_queries_from_keywords(kws)
    queries = []
    for q in (llm_qs + [q.replace("query: ","") for q in base_qs] + ["postoperative drain care", "biliary drain care", "discharge instructions surgery"]):
        q = q.strip()
        if q and q not in queries:
            queries.append(q)
    bm25 = get_bm25_retriever(prefix)

    all_docs, seen = [], set()
    for q in queries:
        faiss_hits = faiss_vs.similarity_search_with_score(f"query: {q}", k=FAISS_CAND_K)
        faiss_ranked = [(d, s) for d, s in faiss_hits]

        bm25_hits = bm25.get_relevant_documents(q)[:BM25_CAND_K]
        bm25_ranked = [(d, None) for d in bm25_hits]

        fused = rrf_fuse([faiss_ranked, bm25_ranked], k=top_per_q)
        fused = rerank_with_cross_encoder(q, fused) if USE_RERANKER else fused

        for d in fused:
            t = _strip_passage_prefix(d.page_content.strip())
            if 60 <= len(t) <= 1200:
                key = (t[:200], d.metadata.get("doc_id"))
                if key not in seen:
                    seen.add(key)
                    all_docs.append(t)
            if len(all_docs) >= max_total:
                break
        if len(all_docs) >= max_total:
            break

    bullets = []
    for t in all_docs:
        for ln in t.splitlines():
            s = ln.strip()
            if not s:
                continue
            if _NON_INSTR_RE.search(s):
                continue
            if s.startswith(("-", "*")) or re.search(r"^(Call|Avoid|Continue|Take|Keep|Walk|Shower|Do not|Don't|Monitor|Check)\b", s, re.I):
                s = re.sub(r"^[-*•]\s*", "- ", s)
                bullets.append(s)
    uniq, bag = [], set()
    for b in bullets:
        if b not in bag:
            bag.add(b); uniq.append(b)
        if len(uniq) >= max_total:
            break
    exemplars = []
    if len(uniq) >= 6:
        exemplars.append("\n".join(uniq[:12]))
    else:
        exemplars.extend(all_docs[:6])
    return exemplars, {"llm_queries": llm_qs, "heuristic": kws}

# =========================
# 7) 프롬프트/생성/후처리 유틸
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

# ---- 패턴들
_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

# ---- DI 섹션 감지
def _extract_di_chunk(note_text: str) -> str:
    if not note_text: return ""
    m = re.search(r"(?is)(Discharge\s+Instructions?|Patient\s+Instructions?|Discharge\s+Plan)\s*:?\s*(.+?)(\n[A-Z][A-Za-z ]{2,40}\s*:|\Z)", note_text)
    return (m.group(2) if m else "").strip()

def has_explicit_di(note_text: str, min_bullets: int = 3) -> bool:
    chunk = _extract_di_chunk(note_text)
    if not chunk: return False
    cnt = 0
    for ln in chunk.splitlines():
        s = ln.strip()
        if not s: continue
        if (s.startswith(("-", "*")) or _DIRECTIVE_RE.search(s)) and not _NON_INSTR_RE.search(s):
            cnt += 1
    return cnt >= min_bullets

# ---- BHC 클린업 (문단 강제, 섹션/해시 제거)
_BHC_SECTION_RE = re.compile(r"(?im)^\s*(TRANSITIONAL ISSUES|ADMISSION DIAGNOSES|ADMISSION TREATMENTS|RESPONSE TO CARE PLAN|Chronic Conditions|Acute Issues|New Acute Issues|New Acute Problems)\s*:.*$")
def cleanup_bhc(text: str) -> str:
    lines = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if s.startswith("#"):
            continue
        if _BHC_SECTION_RE.match(s):
            continue
        if re.match(r"(?i)^(service|date|attending)\s*:", s):
            continue
        lines.append(s)
    # 문단 합치기
    para = " ".join(lines)
    # 공백 정리
    para = re.sub(r"\s{2,}", " ", para).strip()
    # 문장 수 제한(대략적)
    parts = re.split(r"(?<=[\.\!\?])\s+", para)
    if len(parts) > 8:
        para = " ".join(parts[:8])
    return para

# ---- 불릿/줄바꿈 재흐름
def reflow_wrapped_bullets(text: str) -> str:
    out = []
    cur = None
    for ln in text.splitlines():
        s = ln.rstrip()
        if not s:
            continue
        if s.lstrip().startswith("- "):
            if cur:
                out.append(cur.strip())
            cur = s.strip()
        else:
            # 이전 불릿에 이어 붙임
            if cur:
                joiner = " " if not cur.endswith((".", ":", ";", ",")) else " "
                cur = cur + joiner + s.strip()
            else:
                # 불릿이 아니면 스킵
                continue
    if cur:
        out.append(cur.strip())
    # 마침표 보정
    fixed = []
    for b in out:
        b2 = b
        if not re.search(r"[\.!\?]$", b2):
            b2 = b2 + "."
        fixed.append(b2)
    return "\n".join(fixed)

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    if re.search(r"(?m)^\s*-\s+", text):
        text = reflow_wrapped_bullets(text)
        return "\n".join(text.splitlines()[:max_bullets])
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}." if not s.endswith(".") else f"- {s}" for s in sents if not _NON_INSTR_RE.search(s)][:max_bullets]
    return "\n".join(bullets)

def build_di_prompt_with_rag(note_text: str, di_exemplars: list, strict: bool) -> str:
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])
    if strict:
        system = DI_INSTR_STRICT + RAG_STRICT_BLOCK + """
- Output MUST be 5–8 bullet lines, each starting with "- ", and each bullet begins with a directive verb (e.g., "Take", "Continue", "Avoid", "Call").
"""
    else:
        system = DI_INSTR_SOFT + RAG_SOFT_BLOCK
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False

# ---- 수치/단위 가드 (무게/온도/시간/용량 등)
_NUMERIC_GUARD_RE = re.compile(
    r"(?<!\w)(\d+(?:\.\d+)?)\s?(?:cc/?hr|cc|ml/?hr|ml|mg|mcg|g|units?|°f|°c|degrees|hours?|hrs?|days?|weeks?|months?|lb|lbs|pounds?|kg|kilograms?|%)\b",
    re.I
)

# ---- 장치/배액 가드
_DEVICE_TERMS = re.compile(r"\b(ptbd|biliary\s+(drain|catheter)|jp\s*drain|jackson-?pratt|chest\s*tube|foley|peg|nj\s*tube|ng\s*tube|wound\s+vac)\b", re.I)

def numbers_ok(bullet: str, source: str) -> bool:
    """불릿 안의 모든 '숫자+단위' 토큰이 소스에 존재해야 함"""
    found = _NUMERIC_GUARD_RE.findall(bullet)
    if not found:
        return True
    src_norm = re.sub(r"\s+", " ", source).lower()
    for m in _NUMERIC_GUARD_RE.finditer(bullet):
        frag = m.group(0).lower().strip()
        if frag not in src_norm:
            return False
    return True

def device_gate(bullet: str, source: str) -> bool:
    """장치/배액 관련 불릿은 소스에 같은 장치 키워드가 있어야 함"""
    if not _DEVICE_TERMS.search(bullet or ""):
        return True
    return bool(_DEVICE_TERMS.search(source or ""))

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        # 최소/최대 길이 필터로 의미없는 불릿 제거
        if len(b) < 20 or len(b) > 260:
            continue
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

def filter_di_to_source(di_text: str, source: str, min_run: int = 12) -> str:
    keep = []
    for b in di_text.splitlines():
        s = b.strip()
        if not s or not s.startswith("- "): continue
        if not _DIRECTIVE_RE.search(s): continue
        if _NON_INSTR_RE.search(s): continue
        if not has_long_overlap(s, source, min_run=min_run): continue
        if _TRIGGER_START.match(s) and not has_long_overlap(s, source, min_run=max(min_run, 20)): continue
        if _DOSE_PAT.search(s) and not _dose_fragments_in_source(s, source): continue
        if not numbers_ok(s, source): continue       # (NEW) 숫자/단위 가드
        if not device_gate(s, source): continue      # (NEW) 장치/배액 가드
        keep.append(s)
    out, seen = [], set()
    for k in keep:
        if k not in seen: seen.add(k); out.append(k)
    return "\n".join(out)

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        # 과도하게 비임상적/허무맹랑한 표현 제거(웃기/울기 등)
        if re.search(r"\b(laugh|cry|kiss|tickle|massage|caress)\b", ln, re.I):
            continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if has_long_overlap(s_norm, note_text, 10) and s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(하이브리드 RAG)
# =========================
N = len(sample_df)  # 선택된 표본 전체 처리
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = cleanup_bhc(re.sub(r"\n{3,}", "\n\n", bhc_raw).strip())

    # ---- DI: 하이브리드 RAG로 예시 회수 → 프롬프트에 포함 → 생성
    di_exemplars, meta = retrieve_di_exemplars_hybrid(
        note_text, vs, RAG_FAISS_DIR, top_per_q=12, max_total=12
    )
    print(f"[검색요약] LLM_Q={meta['llm_queries']} | HEUR={meta['heuristic']} | 예시 {len(di_exemplars)}개")

    STRICT_MODE = has_explicit_di(note_text)
    di_prompt   = build_di_prompt_with_rag(note_text, di_exemplars, strict=STRICT_MODE)
    di_raw      = _generate_text(trained_model, di_prompt, max_new_tokens=800)

    # ---- DI 후처리
    di_norm = normalize_bullets(ensure_bulleted(di_raw))
    di_norm = reflow_wrapped_bullets(di_norm)  # 줄바꿈 재정렬
    # 불릿 수 제한 및 중복 제거
    uniq = []
    seen = set()
    for b in di_norm.splitlines():
        s = b.strip()
        if not s or s in seen:
            continue
        seen.add(s)
        uniq.append(s)
        if len(uniq) >= 8:
            break
    di_norm = "\n".join(uniq)

    if STRICT_MODE:
        di_text = filter_di_to_source(di_norm, note_text, min_run=8)
        di_text = sanitize_di(di_text)
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "No explicit discharge instructions found in source."
    else:
        # SOFT 모드에서도 숫자/장치 가드 적용
        soft_keep = []
        for b in di_norm.splitlines():
            if not b.strip(): continue
            if not numbers_ok(b, note_text):
                continue
            if not device_gate(b, note_text):
                continue
            soft_keep.append(b)
        di_text = sanitize_di("\n".join(soft_keep))
        di_text = ensure_core_categories(di_text, note_text)
        if not di_text.strip():
            di_text = "\n".join([
                "- Continue your home medications exactly as instructed on your discharge paperwork.",
                "- Walk several times daily and avoid heavy lifting until cleared by your surgeon.",
                "- Keep your incision clean and dry; you may shower but avoid soaking baths or swimming.",
                "- Watch for fever, worsening pain, redness/swelling or drainage from the incision, chest pain, or shortness of breath—seek care or call right away if these occur.",
                "- Schedule and attend follow-up with your surgeon/primary care as instructed.",
                "- Drink fluids and advance diet as tolerated unless told otherwise."
            ])

    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Qwne3사용 코드

In [ ]:
# 충돌 가능 패키지 정리
!pip -q uninstall -y bitsandbytes pandas pyarrow rich requests || true

# Colab 기본과 맞추고, bnb/transformers는 최신으로
!pip -q install \
  "pandas==2.2.2" \
  "pyarrow>=14,<20" \
  "requests==2.32.4" \
  "rich<14" \
  "bitsandbytes>=0.43.3" \
  "transformers>=4.44.2" \
  "accelerate>=0.33.0" \
  "peft>=0.12.0" \
  "trl" \
  "sentence-transformers>=2.5.1" \
  "faiss-cpu==1.8.0.post1" \
  "langchain==0.2.14" "langchain-community==0.2.12"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.1 requires langchain-core>=0.3.67, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.2.4 which is incompatible.


In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U "bitsandbytes>=0.43.1" "transformers>=4.41.0" "accelerate>=0.30.0" "peft>=0.11.0" datasets rich
!pip -q install -U langchain langchain-community faiss-cpu sentence-transformers rank-bm25

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from collections import defaultdict, Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000  # 한 입력 시퀀스(텍스트)를 토큰화/전처리 후 최대로 허용할 길이
SHUFFLE_SEED   = 42     # 데이터 셔플시 재현성을 위한 시드값. 같은 시드를 쓰면 셔플 결과가 항상 동일하게 나옴
TEST_LIMIT     = 3      # 테스트용으로 뽑을 샘플 개수
KEYS           = ["subject_id", "hadm_id", "note_id"]   # 노트를 고유하게 식별하거나 병합할 떄 쓰는 키 컬럼들

# True면 "필터/병합이 끝난 최종 데이터셋"에서 연속된 구간을 그대로 잘라(START_AT_ROW 부터 TEST_LIMIT개)테스트 샘플로 사용, Fals면 기존에 쓰던 무작위 샘플링을 쓰는 형태
USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000   # 1-based index (필터/병합 후 기준)

MODEL_DIR    = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR  = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.4"
BASE_DATA    = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"
KW_MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"  # 키워드 추출용 Qwen3-8B

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) DI 전용 FAISS/E5 경로 ====
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) 모델 로더를 안전하게(4bit→8bit→fp16/bf16 폴백)
# =========================
# _auto_dtype(): GPU를 보고 모델이 계산에 쓸 숫자 형식을 고름
def _auto_dtype():
    # nvidia gpu가 사용 가능한지 확인
    if torch.cuda.is_available():
        # gput의 '연산 능력'버전을 가져
        major, _ = torch.cuda.get_device_capability()
        # major가 8이상인 최신 GPU는 bfloat16연산에 최적화 되어 있어 bfloat16을 반환
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float32

def load_causal_safely(model_dir: str, try_4bit: bool = True):
    dtype = _auto_dtype()
    # 1) VRAM가장 적게 쓰는 4bit 시도
    if try_4bit:
        try:
            # 모델의 파라미터를 4bit정밀도로 불러옴
            bnb_config = BitsAndBytesConfig(
                # nf4라는 고품질 양자화 방식을 사용, bnb_4bit_compute_dtype=dtype: 4bit로 저장된 가중치를 실제 계산할 때
                # _auto_dtype()이 찾아낸 고품질 dtype(bfloat16/float16)으로 변환해서 사용
                # 즉 16bit짜리를 4bit로 압축해서 VRAM에 저장하고, 사용할 때 16bit로 틀어서 손실 최소화 방식
                load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=dtype
            )
            return AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
            )
        except Exception as e:
            print(f"[warn] 4-bit 로드 실패 → 8-bit 시도: {e}")
    # 2) 8bit 시도 -> 4bit 로드 실패시
    try:
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)
        return AutoModelForCausalLM.from_pretrained(
            model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
        )
    except Exception as e:
        print(f"[warn] 8-bit 로드 실패 → fp16/bf16 시도: {e}")
    # 3) 8bit 로드마저 실패하면, 양자화를 포기하고 표준 16bit방식으로 로드
    return AutoModelForCausalLM.from_pretrained(
        model_dir, torch_dtype=dtype, device_map="auto", offload_buffers=True
    )

# ===== Qwen2 + 어댑터 =====
base_model = load_causal_safely(MODEL_DIR, try_4bit=True)  # 앞에서 선언했던 load_casual_safely함수를 통해 VRAM을 절약하며 모델 로드
# 파인튜닝 시킨 어댑터를 플러그인 처럼 장착, PEFT(Parameter-Efficient Fine-Tuning)은 거대한 원본 모델을 통째로 수정하지 않고, 어댑터라는 작은 부품만 추가로 훈련시키는 방식
# .eval()은 '추론 모드'로 설정하여, 훈련이 아닌 실제 요약 생성에 최적화를 시킴
trained_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR, is_trainable=False).eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)   # 모델이 글자를 이해할 수 있도록 토크나이저 로드
# 여러 문장을 동시에 처리(batching)할 때 길이를 맞추기 위해 padding토큰이 필요한데 만약 pad_token이 없으면 문장 끝을 의미하는 eos_token을 대신 사용
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # 길이를 맞추기 위한 패딩 토큰을 오른쪽에 추가
tokenizer.max_seq_length = MAX_SEQ_LENGTH
print("✅ Trained model & tokenizer loaded")

# ===== (NEW) Qwen3-8B 키워드 추출기 =====
kw_model = load_causal_safely(KW_MODEL_DIR, try_4bit=True).eval()
kw_tokenizer = AutoTokenizer.from_pretrained(KW_MODEL_DIR, trust_remote_code=True)  # Qwen3에 맞는 별도의 토크나이저 생성
if kw_tokenizer.pad_token is None:
    kw_tokenizer.pad_token = kw_tokenizer.eos_token
# 왼쪽 패딩을 하는 이유는 배치 생성 시 마지막 <eos>토큰이 같은 열에 모여 있게 되므로 그 위치의 logits을 뽑아 쓰기 수월함
kw_tokenizer.padding_side = "left"
print("✅ Qwen3-8B keyword extractor loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

# 입력과 정답, 두 개의 데이터 프레임을 KEYS(subject_id, hadm_id등)을 기준으로 하나로 합침
# how="inner" => 입력과 정답이 둘 다 존재하는 케이스만 남기겠다는 의미
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")

# 병합된 데이터에 text라는 컬럼이 반드시 존재해야 함을 확인하는 코드. 만약 없으면 오류를 내고 멈춤.
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

# both_empty: 정답 데이터인 bhc_text와 di_text 두 컬럼이 동시에 비어있는 행을 찾음
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
# ~표시는 NOT을 의미. 즉, BHC와 DI가 둘 다 비어있는 행을 제거하고 둘 중 하나라도 정답이 있는 행만 남김
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

# ── 샘플 선택 로직 ─────────────────────────────────────────
if USE_SEQUENTIAL_SLICE:  # 순차 슬라이스 방식
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1  # start_idx = 40000 - 1 즉, 39999이다.
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    # iloc를 사용해 정리된 전체 데이터(test_merged_df)에서 정확히 39999, 40000, 40001번 행을 sample_df로 잘라냄
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    # 기존 무작위 샘플링
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        # SEED값을 이용해 무작위로 뽑되, 실행할 때마다 항상 동일한 3개가 뽑히도록 보장
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")
# ─────────────────────────────────────────────────────────

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the "Brief Hospital Course (BHC)"
for THIS admission using facts explicitly present in the INPUT note.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., "glomerular basement membrane (GBM)").

STYLE & CONTENT
- STYLE: Prefer bullets for scannability; short paragraphs are acceptable.
- ORDER: Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at discharge → pending studies/follow-up items.
- INCLUDE: presenting problem; salient exam/imaging/labs; pivotal decisions and treatments (med changes with stated reasons); clinical response/trends; status at discharge; explicitly listed pending items.
- Keep it clinical, crisp, readable, and de-duplicated.

GUARDRAILS
- Do not invent demographics/PHI (age, phone, MRN, addresses).
- Do not label values as high/low/improved unless the input does so (e.g., *, H, L, or explicit wording).
- Do NOT introduce section headers (e.g., "TRANSITIONAL ISSUES", "ADMISSION DIAGNOSES", etc.).

LENGTH TARGET
- TARGET: ≤ 8 bullets or a short paragraph of comparable density.

OUTPUT
Brief Hospital Course
- Output MUST be 5–8 bullet lines, each starting with "- ".
<content only; DO NOT output any other headings>
"""


# (NEW) 모든 DI 프롬프트를 DI_INSTR_RAG 하나로 통일합니다.
# (기존 DI_INSTR_STRICT, DI_INSTR_SOFT, RAG_STRICT_BLOCK, RAG_SOFT_BLOCK 모두 삭제)
DI_INSTR_RAG = """
You are an expert physician-writer. Your task is to compose a clear, safe, and patient-facing
"Discharge Instructions (DI)" for the CURRENT PATIENT, based on the INPUT NOTE and retrieved examples (RAG context).

GOAL
- Summarize safe and standard discharge instructions that would apply to this patient's condition, treatments, and medications described in the INPUT NOTE.
- Use retrieved DI exemplars only as references for structure, style, and typical categories (not for copying specific data).

CONTENT RULES
- DO NOT copy or re-use any patient identifiers, dates, hospital names, or personal information from exemplars.
- DO NOT invent or alter medication names, doses, frequencies, or appointment dates that are not explicitly present in INPUT.
- You MAY include general, standard, patient-facing guidance suited to the condition/procedure:
  • Activity and physical limits
  • Wound/incision care
  • When to call or return to ER
  • Follow-up and monitoring recommendations
  • Diet, hydration, and lifestyle advice

RAG CONTEXT USAGE
- Retrieved DIs are for style and typical content structure only.
- If any RAG exemplar conflicts with the INPUT NOTE, prefer the INPUT NOTE.
- Generalize exemplar content into safe, de-identified, non-specific advice.

STYLE
- Write 6–8 concise bullet points.
- Each bullet should begin with a directive verb (e.g., Take / Continue / Avoid / Call / Walk / Keep).
- Use simple, reassuring language understandable to patients and families.
- No more than 1–2 sentences per bullet.

OUTPUT FORMAT
- Bulleted text only (no numbered lists, no subheadings).
"""


# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
# LangChain라이브러리를 사용해 Hugging Face의 임베딩 모델을 불러옴
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64}, # "normalize_embeddings": True -> 모든 벡터의 크기를 1로 표준화 함
)
# FAISS: 대용량 벡터를 초고속으로 검색할 수 있는 벡터 검색 라이브러리, load_local: RAG_FAISS_DIR경로에 미리 저장해 둔 벡터 데이터베이스를 불러옴
# emb-> 방금 로드한 E5_MODEL을 FAISS에 전달. 이걸 통해 새로운 검색어를 벡터로 번역한 뒤 검색
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)
retriever = vs.as_retriever(  # as_retriever-> FAISS DB를 LangChain 표준 검색기(retriever)로 포장
    search_type="mmr",        # MMR(Maximal Marginal Relevance)라는 검색 방식을 사용, 검색어와 비섯흐면서도 검색된 결과끼리는 서로 다양한 예시를 뽑아줌
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5} # fetch_k-> 먼저 비슷한 예시 50개 뽑고, k: 8-> 50개 중 다양성을 고려해 8개 최종 선별
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# ==== RAG 파라미터 (보완) ====
# 위에서 한 MMR방식 대신 BM25(키워드 검색)과 FAISS(벡터 검색)를 결합하는 더 고도화 된 하이브리드 검색
FAISS_CAND_K = 50 # FAISS를 통해 50개 후보 가져옴
BM25_CAND_K  = 80 # BM25(키워드 검색)으로 후보 80개 가져옴
FUSE_TOPK    = 12 # 두 검색결과를 RRF라는 기법으로 융합한 뒤, 최종적으로 상위 12개의 예시만 가져옴
USE_RERANKER = False  # 재정렬기 사용여부로써 True이면 융합된 12개의 결과를 Corss-Encoder라는 고성능 모델이 다시 읽어보고 최종 순위를 매겨 정확도를 높임
RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# ==== BM25 문서 로딩/리트리버 (mapping jsonl 필요) ====
# BM25란 jsonl파일을 읽어서 문장 단위로 인덱싱 해두고, 그 문장들을 단어(token)로 분해해서 어떤 단어가 어디 문서(불릿)에 얼마나 많이 등장하는지를 기록해 두는 방식
_BM25 = None
def _load_di_mapping(prefix: str):  # BM25가 검색할 원본 문서를 불러옴
    mapping_path = prefix + ".mapping.jsonl"  # di_index.mapping.jsonl같은 파일을 찾음
    if not os.path.exists(mapping_path):
        alt = os.path.join(os.path.dirname(prefix), "di_index.mapping.jsonl") # 존재하지 않는다면 상위 폴더에서 di_index.mapping.jsonl을 찾음
        if os.path.exists(alt):
            mapping_path = alt
        else:
            raise FileNotFoundError(f"DI mapping jsonl not found: {mapping_path}")
    texts, ids = [], []
    with open(mapping_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            txt = (obj.get("di_text","") or "").strip()
            if not txt:
                continue
            texts.append(txt)
            ids.append(obj.get("doc_id"))
    # Document객체 생성, jsonl파일을 한 줄씩 읽어, di_text는 page_content에, doc_id는 metadata에 담아 LangChain이 표준으로 사용하는 Documnet객체 리스트로 만들어 반환
    docs = [Document(page_content=t, metadata={"doc_id":i}) for t,i in zip(texts, ids)]
    return docs

# _load_di_mapping으로 불러온 문서를 기반으로 실제 검색기(BM25_Retriever)를 생성
def get_bm25_retriever(prefix: str):
    global _BM25  # _BM25라는 전역 변수 사용
    # BM25 리트리버를 초기화하는 작업은 시간이 오래 걸림. 따라서 한번 만들면 _BM25 변수에 저장(캐싱)해두고, 다음 호출부터는 이미 만들어진 것을 즉시 반환하여 성능 최적화
    if _BM25 is not None:
        return _BM25
    docs = _load_di_mapping(prefix)
    bullets = []
    for d in docs:
        for ln in d.page_content.splitlines():  # DI텍스트를 한 줄 단위로 쪼갬
            s = ln.strip()
            if len(s) < 20: # 너무 짧은 줄은 버림
                continue
            if re.search(r"^(Discharge|Service|Date|Attending)\b", s, re.I): # Discharge, Service처럼 지침이 아닌 메타 데이터는 버림
                continue
            bullets.append(Document(page_content=s, metadata=d.metadata)) # 이렇게 정제된 '개별 불릿'들을 새로운 Document List로 만듦
    #  BM25Retriever.from_documents(bullets if bullets else docs) -> 불릿 리스트 기반으로 BM25Retriever를 생성하고 _BM25변수에 캐싱
    _BM25 = BM25Retriever.from_documents(bullets if bullets else docs)
    _BM25.k = BM25_CAND_K
    return _BM25

# ==== BM25 호출 호환 래퍼 (LangChain 구/신버전 모두 지원) ====
# get_bm25_retriever로 만든 검색기를 실행하는 역할
def bm25_query(bm25, query: str, k: int):
    # k 설정(지원 버전에서만)
    # 신버전 API -> .invoke(query)사용
    # 구버전 API -> .get_relevant_documents(query)사용
    try:
        if hasattr(bm25, "k"):
            bm25.k = k
    except Exception:
        pass

    # 신버전: Runnable API
    if hasattr(bm25, "invoke"):
        docs = bm25.invoke(query)
    # 구버전: 표준 메서드
    elif hasattr(bm25, "get_relevant_documents"):
        docs = bm25.get_relevant_documents(query)
    else:
        raise AttributeError("BM25 retriever has neither 'invoke' nor 'get_relevant_documents'.")

    # 상위 k개만 반환
    return (docs or [])[:k]


# ==== RRF 융합 ====
from collections import defaultdict as _dd
# RRF는 BM25, E5(FAISS), 기타 검색 결과를 순위 기반으로 공평하게 합쳐 상위 k개를 뽑
def rrf_fuse(ranklists, k=FUSE_TOPK, c=60): # rrflists-> 여러 개의 랭킹 리스트들의 집합
    agg = _dd(float)  # 각 문서의 'RRF점수'를 합산할 점수판을 만듦
    seen_doc = {}
    for rl in ranklists:  # ranklists를 차례대로 순회
        for rank, (doc, _) in enumerate(rl, start=1): # 각 검색 결과 목록 내에서 1등,2등,3등 문서를 확인
            key = (doc.page_content[:200], doc.metadata.get("doc_id"))
            seen_doc[key] = doc
            agg[key] += 1.0 / (c + rank)  # 문서 점수 알고리즘, 만약 A문서가 FAISS에서 1등, BM25에서 5등이라면 각 분야의 등수별로 점수를 메겨서 합산
    fused = sorted(agg.items(), key=lambda x: x[1], reverse=True)
    fused_docs = [seen_doc[k] for k,_ in fused[:k]]
    return fused_docs

# ==== (옵션) 크로스엔코더 리랭커 ====
# RRF보다 훨씬 더 강하지만 느린 모델을 사용해 RRF가 뽑은 12개의 순위를 다시 매기는 최종 품질 검사 단계
_reranker = None
def _get_reranker():
    global _reranker  # Reranker모델을 한번만 로드하고 캐싱함, BM25와 같은 원리
    if _reranker is not None:
        return _reranker
    try:
        from sentence_transformers import CrossEncoder
        _reranker = CrossEncoder(RERANKER_MODEL)  # ms-marco-MiniLM-L-6-v2같은 Reranker모델을 로드
    except Exception as e:
        print(f"[warn] reranker load failed: {e}")
        _reranker = None
    return _reranker

def rerank_with_cross_encoder(query: str, docs: list):
    if not USE_RERANKER:  # USE_RERANKER = False이므로 이 함수는 현재 비활성화 되어 있음.즉 그냥 RRF결과만 그대로 반환
        return docs
    ce = _get_reranker()
    if ce is None:
        return docs
    pairs = [(query, d.page_content) for d in docs]
    scores = ce.predict(pairs)
    order = np.argsort(scores)[::-1]
    return [docs[i] for i in order]

# =========================
# 6) 키워드 추출(휴리스틱 + LLM)
# =========================
# 이 목록에 있는 단어나 접미사가 포함된 구절은 '의학적 키워드'일 확률이 높다고 가정
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
# '의미 없는 불용어(stop word)필터
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3, 2, 1): # 3단어 구절, 2단어 구절, 1단어 구절 순서로 텍스트를 훑음. 왜냐하면 failure보다 renal failure가 더 구체적이고 좋은 키워드이므로(N-gram)
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            # 탐색한 구절(ph)가 _DI_SUFFIX_HINTS 리스트 단어를 하나라도 포함하고 있는지 확인
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                # 그 구절이 _STOP필터 목록의 단어들로만 이루어져 있지는 않는지 확인
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    # 살아남은 후보 구절들에 대해 두가지 기준으로 점수를 매김
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)} # low.count(ph)-> 빈도, len(ph) -> 길이
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)  # 빈도 1순위, 길이 2순위
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen): # 중복을 방지
            continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms: # max_terms만큼의 최상위 키워드를 리스트로 반환
            break
    return out

# 위에서 추출된 키워드 리스트를 RAG의 E5 임베딩 모델이 알아들을 수 있는 검색어 형식으로 포장
def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]  # intfloat/35-base-v2모델은 텍스트 앞에 'query:'라는 접두사가 붙으면 텍스틀 검색할 문장이 아니라 검색어로 인식
    return qs if qs else ["query: discharge instructions", "query: patient instructions"] # 만약 함수가 키워드를 하나도 찾지 못했을 경우 TAG검색이 실패하지 않도록 기본 검색어 반환

# Qwen3전용 프롬프트 포맷터, LLM모델에게 보낼 지시문을 Qwen3모델이 알아들을 수 있는 채팅 형식의 문자열로 변환
def _kw_serialize_chat(messages: List[dict]) -> str:
    # Qwen3 토크나이저의 내장 함수를 사용해, 이 리스트를 모델이 이해하는 특수 토큰이 포함된 단일 문자열로 변환
    return kw_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode() # 이 데코레이터는 PyTorch에 '지금은 훈련이 아니라 추론중'임을 알리는 데코레이터. 메모리 사용량을 줄이고 속도를 비약적으로 향상시킴
# Qwen3모델을 호출하여 키워드 생성
def _kw_generate_text(model, prompt: str, max_new_tokens: int = 256) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = kw_tokenizer(prompt, return_tensors="pt").to(device) # 프롬프트 문자열을 토큰ID(숫자)로 변환하고 GPU로 보냄
    gen_ids = model.generate( # 텍스트 생성을 시작
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # 랜덤 샘플링을 끔. 모델은 항상 가장 확률이 높은 단어만 선택(매번 결과가 동일하게 나옴). 키워드 추출처럼 창의성이 필요 없는 작업에 필수
        repetition_penalty=1.15,  # 똑같은 단어를 반복하려는 경향에 약간의 패널티를 줌
        no_repeat_ngram_size=6,   # 6단어 이상으로 구성된 구절이 똑같이 반복되는 것을 방지
        eos_token_id=kw_tokenizer.eos_token_id,
        pad_token_id=kw_tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]  # 모델 전체 출력(입력 프롬프트 + 생성된 텍스트)에서 '새로 생성된 텍스트'부분만 깔끔하게 잘라냄
    return kw_tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip() # 토큰 ID를 다시 사람이 읽을 수 있도록 decode

# 휴리스틱 방식에서 썼던 _STOP목록에 LLM이 출력할 만한 새로운 불용어를 추가
_STOP_DISEASE = _STOP | {"patient", "disease", "syndrome", "disorder", "condition", "history"}

# 지저분한 문자열을 깨끗한 파이썬 리스트로 만듦
def _postprocess_kw(cands: List[str], k: int = 5) -> List[str]:
    out, seen = [], set()
    for t in cands:
      # [, ], ", , ->같은 JSON 특수문자를 모두 제거하고 소문자로 바꿈
        s = re.sub(r"[^a-z0-9 \-\/]", " ", str(t).lower()).strip()
        s = re.sub(r"\s{2,}", " ", s)
        if not s or len(s) < 3:
            continue
        if s in _STOP_DISEASE:  # history, disease같은 불용어를 걸러냄
            continue
        # 키워드가 pancreatic disease라면 disease가 불용어라고 해서 전체를 버리는 것은 아까우니 pancreatic만 남김
        s = re.sub(r"\b(disease|disorder|syndrome|condition)\b", "", s).strip()
        s = re.sub(r"\s{2,}", " ", s)
        if not s or s in seen:  # 빈 문자열이 되었거나 이미 리스트에 있는 중복 키워드라면 탈락
            continue
        seen.add(s)
        out.append(s) # 최종적으로 키워드를 리스트에 추가
        if len(out) >= k: # 최대 k개(현재는 5개)키워드만 선택
            break
    return out

def llm_extract_disease_keywords(note_text: str, k: int = 5) -> List[str]:
    """Qwen3-8B에게 질병/진단 키워드를 최대 k개 JSON 배열로 요청. 실패 시 휴리스틱 폴백."""
    if not isinstance(note_text, str) or not note_text.strip():
        return []
    system = (
        "You are a clinical NLP assistant. "# 너는 임상 NLP 조수야
        "From the input discharge note, extract up to 5 DISEASE/CONDITION keywords "  # 질병/상태 키워드 5개 뽑아내
        "(diagnoses, complications, infections, organ failures, named syndromes). "
        "Return a pure JSON array of lowercase strings, no explanations." # 결과는 오직 소문자 JSON배열로만, 설명은 빼고
    )
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nReturn JSON array only." # 출력형식 다시 한번 더 강조
    # _kw_searialize_chat핼퍼 함수를 호출하여, 이 시스템/유저 프롬프트를 Qwen3모델이 이해할 수 있는 채팅 형식의 문자열로 변환
    prompt = _kw_serialize_chat([
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ])
    try:
        raw = _kw_generate_text(kw_model, prompt, max_new_tokens=256) # _kw_generate_text를 실행하여 모델을 실행하고 텍스트를 받음
        # LLM이 지시를 어기고 'Here is th list'처럼 사족을 붙일 경우를 대비해 정규식으로 JSON배열 부분만 정확히 찾아냄
        m = re.search(r"\[[\s\S]*\]", raw)
        arr = json.loads(m.group(0)) if m else json.loads(raw)  # m찾으면 찾은 부분만 파싱, 못 찾으면 전체 텍스트를 파싱 시도
        if not isinstance(arr, list):
            raise ValueError("not a list")
          # LLM이 반환한 리스트(arr)를 _postprocess_kw에 보내 불용어 제거, 만약 빈 리스트가 되었다면 휴리스틱 함수를 대신 실행
        return _postprocess_kw(arr, k=k) or extract_disease_keywords_heuristic(note_text, max_terms=k)
    except Exception as e:
        print(f"[warn] keyword extraction via Qwen3-8B failed: {e}")  # try블록에서 어떤 종류의 오류라도 발생하면 이 코드가 실행
        return extract_disease_keywords_heuristic(note_text, max_terms=k) # LLM추출이 실패했음을 알리고 RAG 파이프 라인 전체가 멈추지 않도록 휴리스틱 함수 호출

# =========================
# 6-1) 하이브리드 RAG: Qwen3-8B 키워드 기반 + BM25 + FAISS + RRF (+옵션 리랭커)
# =========================
def retrieve_di_exemplars_hybrid(note_text: str, faiss_vs, prefix: str, top_per_q: int = 12, max_total: int = 12):
    # ==== 핵심: 질병 키워드 5개를 Qwen3-8B로 추출 ====
    kws = llm_extract_disease_keywords(note_text, k=5)  # qwen3가 키워드 5개 추출
    base_qs = build_e5_queries_from_keywords(kws)       #

    # 간단 동의어 확장
    syn = {
        "ptbd": ["biliary drain", "biliary catheter", "percutaneous transhepatic biliary drain"],
        "pancreatitis": ["acute pancreatitis", "necrotizing pancreatitis"],
        "pneumonia": ["aspiration pneumonia", "community acquired pneumonia"],
        "sepsis": ["septic shock", "bacteremia"]
    }

    queries = []
    for q in [q.replace("query: ","") for q in base_qs]:
        q = q.strip().lower()
        if not q:
            continue
        queries.append(q)
        for kword, exp in syn.items():
            if kword in q:
                queries.extend(exp)

    # 확장된 키워드 리스트(queries)에서 중복을 제거하고 검색어 목록(uniq_q)을 완성
    uniq_q, seen = [], set()
    for q in queries:
        if q not in seen:
            seen.add(q)
            uniq_q.append(q)

    bm25 = get_bm25_retriever(prefix)

    all_docs, seen_doc = [], set()
    for q in uniq_q:  # 모든 검색어에 대해 개별적으로 아래의 검색+융합 과정을 반복 실행
        # 벡터 검색(FAISS, 의미 기반)
        faiss_hits = faiss_vs.similarity_search_with_score(f"query: {q}", k=FAISS_CAND_K)
        faiss_ranked = [(d, s) for d, s in faiss_hits]

        # 키워드 검색(BM25기반)
        bm25_hits = bm25_query(bm25, q, k=BM25_CAND_K)
        bm25_ranked = [(d, None) for d in bm25_hits]

        # 결과 융합 -> RRF함수를 호출
        fused = rrf_fuse([faiss_ranked, bm25_ranked], k=top_per_q)
        # Rerank이용 -> 현재는 사용 안하는중
        fused = rerank_with_cross_encoder(q, fused) if USE_RERANKER else fused

        # 융합된 결과(fused)를 all_docs라는 최종 리스트에 모음
        for d in fused:
            t = _strip_passage_prefix(d.page_content.strip()) # 임베딩용 접두사 'passage:' 제거
            if 60 <= len(t) <= 1200:  # 검색된 예시가 너무 짧거나 길면 버림
                key = (t[:200], d.metadata.get("doc_id"))
                if key not in seen_doc: # 문서 중복 제거, 이미 추가된 예시 DI는 아닌지 확인
                    seen_doc.add(key)
                    all_docs.append(t)
            if len(all_docs) >= max_total:  # 최종 예시(12개)가 모이면 다른 키워드로 검색하지 않고 즉시 루프 탈출
                break
        if len(all_docs) >= max_total:
            break

    # LLM에게 긴 DI 예시 문서를 통쨰로 주는 것보다, 예시에서 가장 핵심적인 불릿만 뽑아서 주는것이 더 효율적임
    bullets = []
    for t in all_docs:  # 수집된 12개의 DI예시 문서를 순회
        for ln in t.splitlines(): # 문서를 다시 한 줄씩 쪼갬
            s = ln.strip()
            if not s:
                continue
            if _NON_INSTR_RE.search(s): # "Findings:", "CT shows.."같은 지침이 아닌 줄은 버림
                continue
            # 진짜 지침만 필터링, Call Avoid같은 지시 동사로 시작하는 줄
            if s.startswith(("-", "*")) or re.search(r"^(Call|Avoid|Continue|Take|Keep|Walk|Shower|Do not|Don't|Monitor|Check)\b", s, re.I):
                s = re.sub(r"^[-*•]\s*", "- ", s)
                bullets.append(s) # 핵심 불릿만 bullets리스트에 모음
    uniq, bag = [], set() # 추출된 불릿들 사이의 중복을 마지막으로 제거
    for b in bullets:
        if b not in bag:
            bag.add(b); uniq.append(b)
        if len(uniq) >= max_total:
            break
    exemplars = []
    if len(uniq) >= 6:
        exemplars.append("\n".join(uniq[:12]))  # 6개 이상의 깨끗한 불릿을 찾았다면(최대 12개) 추가
    else:
        exemplars.extend(all_docs[:6])  # 5개 이하라면 그냥 원본 문서 6개를 줌

    meta = {"llm_keywords": kws}
    return exemplars, meta  # exemplars-> LLM에게 전달할 참고 자료, meta -> 디버깅을 위해 검색에 사용된 Qwen3키워드를 함께 반환

# =========================
# 7) 프롬프트/생성/후처리 유틸
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 실제 LLM 추론 엔진
@torch.inference_mode() # 추론단계이니 기울기 계산 등 불필요한 작업을 모두 생략
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen_ids = model.generate( # 모델의 텍스트 생성 기능 호출
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # 창의적인 답변 생성하지 않도록
        repetition_penalty=1.20, # 같은말 반복현상 방지
        no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]  # 새로 생성된 텍스트 부분만 잘라냄
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

# BHC_INSTR과 note_text원본을 받아서 _searialize_chat을 이용해 _generate_text에 바로 넣을 수 있는 BHC프롬프트 만듦
def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

# ---- 패턴들
# 지시 도사 목록
_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
# 필터 목
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)


# ---- BHC 클린업 (문단 강제, 섹션/해시 제거)
_BHC_SECTION_RE = re.compile(r"(?im)^\s*(TRANSITIONAL ISSUES|ADMISSION DIAGNOSES|ADMISSION TREATMENTS|RESPONSE TO CARE PLAN|Chronic Conditions|Acute Issues|New Acute Issues|New Acute Problems)\s*:.*$")
def cleanup_bhc(text: str) -> str:
    lines = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if s.startswith("#"):
            continue
        if _BHC_SECTION_RE.match(s):
            continue
        if re.match(r"(?i)^(service|date|attending)\s*:", s):
            continue
        lines.append(s)
    # 문단 합치기
    para = " ".join(lines)
    # 공백 정리
    para = re.sub(r"\s{2,}", " ", para).strip()
    # 문장 수 제한(대략적)
    parts = re.split(r"(?<=[\.\!\?])\s+", para)
    if len(parts) > 8:
        para = " ".join(parts[:8])
    return para

# ---- 불릿/줄바꿈 재흐름
def reflow_wrapped_bullets(text: str) -> str:
    out = []
    cur = None
    for ln in text.splitlines():
        s = ln.rstrip()
        if not s:
            continue
        # -로 시작하지 않는 줄을 앞줄에 자동으로 병합
        if s.lstrip().startswith("- "):
            if cur:
                out.append(cur.strip())
            cur = s.strip()
        else:
            if cur:
                # 구두점 추가-> 병합된 불릿의 끝에 마침표나 물음표가 없으면 구두점 추가
                joiner = " " if not cur.endswith((".", ":", ";", ",")) else " "
                cur = cur + joiner + s.strip()
            else:
                continue
    if cur:
        out.append(cur.strip())
    fixed = []
    for b in out:
        b2 = b
        if not re.search(r"[\.!\?]$", b2):
            b2 = b2 + "."
        fixed.append(b2)
    return "\n".join(fixed)

# LLM의 출력이 반드시 불릿 목록 형식이 되도록 보장
def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    if re.search(r"(?m)^\s*-\s+", text):  # 출력이 이미 불릿인 경우
        text = reflow_wrapped_bullets(text) # reflow_wrapped_bullets함수를 호출해 위의 함수에서 청소 작업만 수행
        return "\n".join(text.splitlines()[:max_bullets]) # 최대 8개의 불릿을 반환
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    # 문단을 .단위로 쪼갬, _NON_INSTR_RE필터로 지침이 아닌 ㅁ누장 걸러
    bullets = [f"- {s}." if not s.endswith(".") else f"- {s}" for s in sents if not _NON_INSTR_RE.search(s)][:max_bullets]
    return "\n".join(bullets)

# 최종적으로 DI생성 LLM에게 보낼 프롬프트를 조립
def build_di_prompt_with_rag(note_text: str, di_exemplars: list) -> str:
    """[수정됨] RAG 예시와 노트를 기반으로 DI 프롬프트를 빌드합니다. (단일 DI_INSTR_RAG 사용)"""
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])

    # 항상 새로운 DI_INSTR_RAG 프롬프트를 system 프롬프트로 사용
    system = DI_INSTR_RAG

    # 유저 프롬프트에 (A)원본 노트와 (B)RAG 예시를 모두 포함
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

# 최종 출력물 조립 -> BHC요약 결과물과 DI요약 결과물을 받아 사용자에게 '최종 보고서'형식으로 합침
def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course") # 제목 추가
    out.append(bhc_text.strip())  # BHC 내용 추가
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":  # DI텍스트가 비어있지않고 not found메시지가 아닌지 확인
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")  # 제목 추가
        out.append(di_text.strip()) # 내용 추가
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

"""
# 텍스트 중복 검사기 -> line(LLM이 생성한 줄)이 source(원본 노트)내용과 조금이라도 겹치는지 확
def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool: # 윈도우 크기를 8글자로 설정
    line_norm = re.sub(r'\s+', ' ', line).strip().lower() # 이 조각들 중 하나라도 거대한 source텍스트 어딘가에 존재한다면 True
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False
"""

# ---- 숫자와 단위가 결합된 모든 패턴을 찾음
_NUMERIC_GUARD_RE = re.compile(
    r"(?<!\w)(\d+(?:\.\d+)?)\s?(?:cc/?hr|cc|ml/?hr|ml|mg|mcg|g|units?|°f|°c|degrees|hours?|hrs?|days?|weeks?|months?|lb|lbs|pounds?|kg|kilograms?|%)\b",
    re.I
)
# ---- 특정 의료 장비/배액관을 찾음
_DEVICE_TERMS = re.compile(r"\b(ptbd|biliary\s+(drain|catheter)|jp\s*drain|jackson-?pratt|chest\s*tube|foley|peg|nj\s*tube|ng\s*tube|wound\s+vac)\b", re.I)

# LLM이 생성한 불릿에 '감시목록'의 숫자/단위가 포함 되어 있는가
def numbers_ok(bullet: str, source: str) -> bool:
    found = _NUMERIC_GUARD_RE.findall(bullet) # 원본에 있는가
    if not found: # 없으면 환각으로 간주하고 폐기
        return True
    src_norm = re.sub(r"\s+", " ", source).lower()
    for m in _NUMERIC_GUARD_RE.finditer(bullet):
        frag = m.group(0).lower().strip()
        if frag not in src_norm:
            return False
    return True

# LLM모델이 생성한 불릿에 의료 장비 있는지 확인
def device_gate(bullet: str, source: str) -> bool:
    if not _DEVICE_TERMS.search(bullet or ""):
        return True
    return bool(_DEVICE_TERMS.search(source or ""))

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

# 출력 형식 표준화 => LLM이 생성한 텍스트를 일관된 불릿으로 표준화함
def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s): # 같은 지침이 아닌 줄을 제거
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s): # -가 빠졌으면 강제로 -붙여줌
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if len(b) < 20 or len(b) > 260:
            continue
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

# 의학적 안전을 위한 두 번째 필터
def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue # 명백하게 위험한 지침이 생성되면, 그 줄을 통째로 삭제
        if re.search(r"\b(laugh|cry|kiss|tickle|massage|caress)\b", ln, re.I):
            continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS: # LLM이 실수로 "avoid aspirin"(아스피린 금지)와 "low dose aspirin(아스피린 복용)" 처럼 서로 모순되는 지침을 도잇에 생성했는지 확인
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I) # 만약 발견되면, 환자의 혼란을 막기위한 지시 추가
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

# AI가 생성한 DI가 완전한지 확인하고 빠진 내용이 있다면 원본 노트에서 '추출'하여 보완함
def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)     # 상처
    if not has_activity: need.append(_ACTIVITY_KW)  # 활동
    if not has_emerg:    need.append(_EMERG_KW)     # 응급

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(하이브리드 RAG)
# =========================
N = len(sample_df)  # 선택된 표본 전체 처리
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = cleanup_bhc(re.sub(r"\n{3,}", "\n\n", bhc_raw).strip())

    # ---- DI: 하이브리드 RAG로 예시 회수 → 프롬프트에 포함 → 생성
    di_exemplars, meta = retrieve_di_exemplars_hybrid(
        note_text, vs, RAG_FAISS_DIR, top_per_q=12, max_total=12
    )
    print(f"[검색요약] LLM_KW={meta.get('llm_keywords')} | 예시 {len(di_exemplars)}개")

    di_prompt   = build_di_prompt_with_rag(note_text, di_exemplars)
    di_raw      = _generate_text(trained_model, di_prompt, max_new_tokens=800)

    # ---- DI 후처리
    di_norm = normalize_bullets(ensure_bulleted(di_raw))
    di_norm = reflow_wrapped_bullets(di_norm)  # 줄바꿈 재정렬
    # 불릿 수 제한 및 중복 제거
    uniq = []
    seen = set()
    for b in di_norm.splitlines():
        s = b.strip()
        if not s or s in seen:
            continue
        seen.add(s)
        uniq.append(s)
        if len(uniq) >= 8:
            break
    di_norm = "\n".join(uniq)

    safe_keep = []
    for b in di_norm.splitlines():
        if not b.strip(): continue
        if not numbers_ok(b, note_text):
            continue
        if not device_gate(b, note_text):
            continue
        safe_keep.append(b)
    di_text = sanitize_di("\n".join(safe_keep))
    di_text = ensure_core_categories(di_text, note_text)
    if not di_text.strip():
      di_text = "No explicit discharge instructions found in source"

    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


<>:736: SyntaxWarning: invalid escape sequence '\s'
<>:736: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1633778304.py:736: SyntaxWarning: invalid escape sequence '\s'
  line_norm = re.sub(r'\s+', ' ', line).strip().lower() # 이 조각들 중 하나라도 거대한 source텍스트 어딘가에 존재한다면 True


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[warn] 4-bit 로드 실패 → 8-bit 시도: No package metadata was found for bitsandbytes
[warn] 8-bit 로드 실패 → fp16/bf16 시도: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
[warn] 4-bit 로드 실패 → 8-bit 시도: No package metadata was found for bitsandbytes
[warn] 8-bit 로드 실패 → fp16/bf16 시도: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Qwen3-8B keyword extractor loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 선택 모드: 순차 슬라이스 | 시작(1-based): 40,000 | 선택 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[검색요약] LLM_KW=['failure to thrive', 'bile leak', 'duodenal stricture', 'gallbladder remnant', 'splenomegaly'] | 예시 1개


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
Brief Hospital Course This is a ___ y/o woman who presented with failure to thrive after having a PTBD placed at ___ for a bile leak following a failed ERCP. Her diet was advanced slowly and she tolerated well. The PTBD was removed and she continued to have good oral intake. At the time of transfer, she was tolerating 75 cc/hr of TPN and had been taking some food orally. She was transferred back to ___. She underwent an over-the-wire cholangiography under fluoroscopic guidance where a new 10 ___ cathater was exchanged into the existing PTBD site. There was no evidence 

## 콴에 RAG사용

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U "bitsandbytes>=0.43.1" "transformers>=4.41.0" "accelerate>=0.30.0" "peft>=0.11.0" datasets rich
!pip -q install -U langchain langchain-community faiss-cpu sentence-transformers rank-bm25

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000

# (하나만 씀) — Qwen3-8B로 키워드 추출 + BHC/DI 생성 모두
GEN_MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"

BASE_DATA    = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"
TEST_INPUT   = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT  = f"{BASE_DATA}/discharge_test_output.parquet"

# (RAG) DI 전용
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"

# =========================
# 2) 모델 로더 (4bit→8bit→fp16/bf16 폴백)
# =========================
def _auto_dtype():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float32

def load_causal_safely(model_dir: str, try_4bit: bool = True):
    dtype = _auto_dtype()
    if try_4bit:
        try:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=dtype
            )
            return AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
            )
        except Exception as e:
            print(f"[warn] 4-bit 로드 실패 → 8-bit 시도: {e}")
    try:
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)
        return AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
        )
    except Exception as e:
        print(f"[warn] 8-bit 로드 실패 → fp16/bf16 시도: {e}")
    return AutoModelForCausalLM.from_pretrained(
        model_dir, torch_dtype=dtype, device_map="auto", offload_buffers=True
    )

# ===== Qwen3-8B (생성+키워드 공용) =====
gen_model = load_causal_safely(GEN_MODEL_DIR, try_4bit=True).eval()
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_DIR, trust_remote_code=True)
if gen_tokenizer.pad_token is None:
    gen_tokenizer.pad_token = gen_tokenizer.eos_token
gen_tokenizer.padding_side = "left"
gen_tokenizer.model_max_length = MAX_SEQ_LENGTH
print("✅ Qwen3-8B (generator & keyword) loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if USE_SEQUENTIAL_SLICE:
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명) — 메타/사고 금지 가드 추가
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the "Brief Hospital Course (BHC)"
for THIS admission using facts explicitly present in the INPUT note.
Do NOT include any meta-comments, chain-of-thought, reasoning steps, or apologies.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., "glomerular basement membrane (GBM)").

STYLE & CONTENT
- STYLE: Use concise bullets for scannability.
- ORDER: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at discharge → pending items.
- INCLUDE: presenting problem; salient exam/imaging/labs; pivotal decisions and treatments; trends/response; discharge status; explicit pending items.

GUARDRAILS
- Do not invent demographics/PHI.
- Do not label values as high/low/improved unless stated.

LENGTH TARGET
- TARGET: 5–8 bullets.

OUTPUT
Brief Hospital Course
- Output MUST be 5–8 bullet lines, each starting with "- ".
<content only; DO NOT output any other headings>
"""

DI_INSTR_RAG = """
You are an expert physician-writer. Compose a clear, safe, patient-facing
"Discharge Instructions (DI)" for the CURRENT PATIENT, based on the INPUT NOTE and retrieved examples (RAG context).
Do NOT include any meta-comments, chain-of-thought, reasoning steps, or apologies.

GOAL
- Provide safe, standard instructions suited to this patient's condition/treatments in the INPUT NOTE.
- Use exemplars only for structure/style/categories (do not copy identifiers or specifics).

CONTENT RULES
- Do NOT invent or alter medication names/doses/frequencies/appointments not present in INPUT.
- You MAY include general guidance:
  • Activity/physical limits
  • Wound/incision care
  • When to call/return to ER
  • Follow-up/monitoring
  • Diet/hydration/lifestyle

RAG CONTEXT USAGE
- If any exemplar conflicts with INPUT NOTE, prefer the INPUT NOTE.
- Generalize into safe, de-identified advice.

STYLE
- Write 6–8 concise bullet points.
- Each bullet begins with a directive verb (e.g., Take / Continue / Avoid / Call / Walk / Keep).
- Use simple language.

OUTPUT FORMAT
- Bulleted text only (no numbers, no subheadings).
"""

# =========================
# 5) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

FAISS_CAND_K = 50
BM25_CAND_K  = 80
FUSE_TOPK    = 12
USE_RERANKER = False
RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

_BM25 = None
def _load_di_mapping(prefix: str):
    mapping_path = prefix + ".mapping.jsonl"
    if not os.path.exists(mapping_path):
        alt = os.path.join(os.path.dirname(prefix), "di_index.mapping.jsonl")
        if os.path.exists(alt): mapping_path = alt
        else: raise FileNotFoundError(f"DI mapping jsonl not found: {mapping_path}")
    texts, ids = [], []
    with open(mapping_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            txt = (obj.get("di_text","") or "").strip()
            if not txt: continue
            texts.append(txt); ids.append(obj.get("doc_id"))
    return [Document(page_content=t, metadata={"doc_id":i}) for t,i in zip(texts, ids)]

def get_bm25_retriever(prefix: str):
    global _BM25
    if _BM25 is not None: return _BM25
    docs = _load_di_mapping(prefix)
    bullets = []
    for d in docs:
        for ln in d.page_content.splitlines():
            s = ln.strip()
            if len(s) < 20: continue
            if re.search(r"^(Discharge|Service|Date|Attending)\b", s, re.I): continue
            bullets.append(Document(page_content=s, metadata=d.metadata))
    _BM25 = BM25Retriever.from_documents(bullets if bullets else docs)
    _BM25.k = BM25_CAND_K
    return _BM25

def bm25_query(bm25, query: str, k: int):
    try:
        if hasattr(bm25, "k"): bm25.k = k
    except Exception:
        pass
    if hasattr(bm25, "invoke"): docs = bm25.invoke(query)
    elif hasattr(bm25, "get_relevant_documents"): docs = bm25.get_relevant_documents(query)
    else: raise AttributeError("BM25 retriever has neither 'invoke' nor 'get_relevant_documents'.")
    return (docs or [])[:k]

from collections import defaultdict as _dd
def rrf_fuse(ranklists, k=FUSE_TOPK, c=60):
    agg = _dd(float); seen_doc = {}
    for rl in ranklists:
        for rank, (doc, _) in enumerate(rl, start=1):
            key = (doc.page_content[:200], doc.metadata.get("doc_id"))
            seen_doc[key] = doc
            agg[key] += 1.0 / (c + rank)
    fused = sorted(agg.items(), key=lambda x: x[1], reverse=True)
    return [seen_doc[k] for k,_ in fused[:k]]

# =========================
# 6) 키워드 추출(휴리스틱 + LLM: Qwen3-8B 재사용)
# =========================
_DI_SUFFIX_HINTS = (
    "itis","oma","emia","osis","pathy","plegia","algia","ism","ia",
    "syndrome","disease","deficiency","failure","injury","infection",
    "sepsis","pneumonia","hypertension","diabetes","cancer","fracture",
    "shock","infarct","stroke","thrombosis","embolism","ulcer","anemia",
    "renal","hepatic","liver","kidney","heart","cardiac","pulmonary","asthma"
)
_STOP = set("""
a an the of and to in on for with without into by from as is are was were be been being
this that these those he she they patient hospital course brief discharge instruction
follow up plan day days present past history acute chronic severe mild moderate during
prior after before within under over abnormal normal lab labs dose prn qd bid tid qid
po iv im subcut x-ray ct mri cbc bmp cmp wnl symptoms signs fever pain cough
diagnosis history problem issue situation
""".split())

def extract_disease_keywords_heuristic(text: str, max_terms: int = 8) -> List[str]:
    if not isinstance(text, str) or not text.strip(): return []
    clean = re.sub(r"[^A-Za-z0-9 /\-]", " ", text)
    toks  = [t.lower() for t in re.split(r"\s+", clean) if t]
    cands = []
    for n in (3,2,1):
        for i in range(len(toks)-n+1):
            ph = " ".join(toks[i:i+n])
            if any(h in ph for h in _DI_SUFFIX_HINTS):
                if not all(w in _STOP for w in ph.split()):
                    cands.append(ph)
    low = clean.lower()
    score = {ph: (low.count(ph), len(ph)) for ph in set(cands)}
    ranked = sorted(score, key=lambda k: (score[k][0], score[k][1]), reverse=True)
    out, seen = [], []
    for ph in ranked:
        if any(ph in s or s in ph for s in seen): continue
        out.append(ph); seen.append(ph)
        if len(out) >= max_terms: break
    return out

def build_e5_queries_from_keywords(kws: List[str]) -> List[str]:
    qs = [f"query: {k}" for k in kws if k.strip()]
    return qs if qs else ["query: discharge instructions", "query: patient instructions"]

def _serialize_chat(messages: List[dict]) -> str:
    return gen_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, tokenizer, prompt: str, max_new_tokens: int = 800) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=1.20,
        no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

_STOP_DISEASE = _STOP | {"patient", "disease", "syndrome", "disorder", "condition", "history"}
def _postprocess_kw(cands: List[str], k: int = 5) -> List[str]:
    out, seen = [], set()
    for t in cands:
        s = re.sub(r"[^a-z0-9 \-\/]", " ", str(t).lower()).strip()
        s = re.sub(r"\s{2,}", " ", s)
        if not s or len(s) < 3: continue
        if s in _STOP_DISEASE: continue
        s = re.sub(r"\b(disease|disorder|syndrome|condition)\b", "", s).strip()
        s = re.sub(r"\s{2,}", " ", s)
        if not s or s in seen: continue
        seen.add(s); out.append(s)
        if len(out) >= k: break
    return out

def llm_extract_disease_keywords(note_text: str, k: int = 5) -> List[str]:
    if not isinstance(note_text, str) or not note_text.strip(): return []
    system = (
        "You are a clinical NLP assistant. "
        "From the input discharge note, extract up to 5 DISEASE/CONDITION keywords "
        "(diagnoses, complications, infections, organ failures, named syndromes). "
        "Return a pure JSON array of lowercase strings, no explanations."
    )
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nReturn JSON array only."
    prompt = _serialize_chat([
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ])
    try:
        raw = _generate_text(gen_model, gen_tokenizer, prompt, max_new_tokens=256)
        m = re.search(r"\[[\s\S]*\]", raw)
        arr = json.loads(m.group(0)) if m else json.loads(raw)
        if not isinstance(arr, list): raise ValueError("not a list")
        return _postprocess_kw(arr, k=k) or extract_disease_keywords_heuristic(note_text, max_terms=k)
    except Exception as e:
        print(f"[warn] keyword extraction via Qwen3-8B failed: {e}")
        return extract_disease_keywords_heuristic(note_text, max_terms=k)

# =========================
# 6-1) 하이브리드 RAG
# =========================
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def retrieve_di_exemplars_hybrid(note_text: str, faiss_vs, prefix: str, top_per_q: int = 12, max_total: int = 12):
    kws = llm_extract_disease_keywords(note_text, k=5)
    base_qs = build_e5_queries_from_keywords(kws)

    syn = {
        "ptbd": ["biliary drain", "biliary catheter", "percutaneous transhepatic biliary drain"],
        "pancreatitis": ["acute pancreatitis", "necrotizing pancreatitis"],
        "pneumonia": ["aspiration pneumonia", "community acquired pneumonia"],
        "sepsis": ["septic shock", "bacteremia"]
    }

    queries, seen = [], set()
    for q in [q.replace("query: ","") for q in base_qs]:
        q = q.strip().lower()
        if not q: continue
        cand = [q]
        for kword, exp in syn.items():
            if kword in q: cand.extend(exp)
        for c in cand:
            if c not in seen: seen.add(c); queries.append(c)

    bm25 = get_bm25_retriever(prefix)

    all_docs, seen_doc = [], set()
    for q in queries:
        faiss_hits = faiss_vs.similarity_search_with_score(f"query: {q}", k=FAISS_CAND_K)
        faiss_ranked = [(d, s) for d, s in faiss_hits]
        bm25_hits = bm25_query(bm25, q, k=BM25_CAND_K)
        bm25_ranked = [(d, None) for d in bm25_hits]
        fused = rrf_fuse([faiss_ranked, bm25_ranked], k=top_per_q)

        for d in fused:
            t = _strip_passage_prefix(d.page_content.strip())
            if 60 <= len(t) <= 1200:
                key = (t[:200], d.metadata.get("doc_id"))
                if key not in seen_doc:
                    seen_doc.add(key); all_docs.append(t)
            if len(all_docs) >= max_total: break
        if len(all_docs) >= max_total: break

    bullets = []
    for t in all_docs:
        for ln in t.splitlines():
            s = ln.strip()
            if not s or _NON_INSTR_RE.search(s): continue
            if s.startswith(("-", "*")) or re.search(r"^(Call|Avoid|Continue|Take|Keep|Walk|Shower|Do not|Don't|Monitor|Check)\b", s, re.I):
                s = re.sub(r"^[-*•]\s*", "- ", s)
                bullets.append(s)
    uniq, bag = [], set()
    for b in bullets:
        if b not in bag:
            bag.add(b); uniq.append(b)
        if len(uniq) >= max_total: break
    exemplars = []
    if len(uniq) >= 6:
        exemplars.append("\n".join(uniq[:12]))
    else:
        exemplars.extend(all_docs[:6])

    meta = {"llm_keywords": kws}
    return exemplars, meta

# =========================
# 7) 프롬프트/생성/후처리 유틸 (+ 사고/메타 제거)
# =========================
def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)

# --- 사고/메타 문장 제거 정규식 & 함수 ---
_THINKING_META_RE = re.compile(
    r"(?i)\b("
    r"let me (think|start|explain)|"
    r"i (will|need to|am going to|think)|"
    r"first(ly)?|second(ly)?|to start|step[s]?:|"
    r"analysis|analysing|reasoning|thought process|chain[- ]?of[- ]?thought|rationale|"
    r"here('?s| is) (how|what)|"
    r"as an ai|as a language model|"
    r"meta(?:-)?comment|"
    r"okay,? let me|let'?s (begin|start)"
    r")\b"
)

def strip_thinking_meta(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s:
            continue
        if re.search(_THINKING_META_RE, s):
            continue
        s = re.sub(_THINKING_META_RE, "", s)
        s = re.sub(r"\s{2,}", " ", s).strip(" -:;")
        if s:
            out.append(s)
    return "\n".join(out)

def reflow_wrapped_bullets(text: str) -> str:
    out = []; cur = None
    for ln in text.splitlines():
        s = ln.rstrip()
        if not s: continue
        if s.lstrip().startswith("- "):
            if cur: out.append(cur.strip())
            cur = s.strip()
        else:
            if cur:
                joiner = " " if not cur.endswith((".", ":", ";", ",")) else " "
                cur = cur + joiner + s.strip()
            else:
                continue
    if cur: out.append(cur.strip())
    fixed = []
    for b in out:
        b2 = b
        if not re.search(r"[\.!\?]$", b2): b2 = b2 + "."
        fixed.append(b2)
    return "\n".join(fixed)

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    if re.search(r"(?m)^\s*-\s+", text):
        text = reflow_wrapped_bullets(text)
        return "\n".join(text.splitlines()[:max_bullets])
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}." if not s.endswith(".") else f"- {s}" for s in sents][:max_bullets]
    return "\n".join(bullets)

# ---- BHC 클린업 (불릿 강제 & 헤더 제거)
_BHC_SECTION_RE = re.compile(r"(?im)^\s*(TRANSITIONAL ISSUES|ADMISSION DIAGNOSES|ADMISSION TREATMENTS|RESPONSE TO CARE PLAN|Chronic Conditions|Acute Issues|New Acute Issues|New Acute Problems)\s*:.*$")
def cleanup_bhc_bulleted(text: str) -> str:
    lines = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if s.startswith("#"): continue
        if _BHC_SECTION_RE.match(s): continue
        if re.match(r"(?i)^(service|date|attending)\s*:", s): continue
        if re.match(r"(?i)^brief hospital course\b", s): continue
        lines.append(s)
    merged = "\n".join(lines)
    bul = ensure_bulleted(merged, max_bullets=8)
    return "\n".join([b for b in bul.splitlines() if b.strip()][:8])

# ---- DI 전용 유틸
def build_di_prompt_with_rag(note_text: str, di_exemplars: list) -> str:
    exemplars = "\n\n".join([f"### DI exemplar #{i+1}\n{_trim(t, 800)}" for i, t in enumerate(di_exemplars)])
    system = DI_INSTR_RAG
    user = f"INPUT NOTE:\n{note_text.strip()}\n\nRetrieved DI Exemplars:\n{exemplars if exemplars else '(none)'}\n"
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s): continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if len(b) < 20 or len(b) > 260: continue
        if b not in seen:
            seen.add(b); final.append(b)
    return "\n".join(final)

_NUMERIC_GUARD_RE = re.compile(
    r"(?<!\w)(\d+(?:\.\d+)?)\s?(?:cc/?hr|cc|ml/?hr|ml|mg|mcg|g|units?|°f|°c|degrees|hours?|hrs?|days?|weeks?|months?|lb|lbs|pounds?|kg|kilograms?|%)\b",
    re.I
)
_DEVICE_TERMS = re.compile(r"\b(ptbd|biliary\s+(drain|catheter)|jp\s*drain|jackson-?pratt|chest\s*tube|foley|peg|nj\s*tube|ng\s*tube|wound\s+vac)\b", re.I)

def numbers_ok(bullet: str, source: str) -> bool:
    found = _NUMERIC_GUARD_RE.findall(bullet)
    if not found: return True
    src_norm = re.sub(r"\s+", " ", source).lower()
    for m in _NUMERIC_GUARD_RE.finditer(bullet):
        frag = m.group(0).lower().strip()
        if frag not in src_norm: return False
    return True

def device_gate(bullet: str, source: str) -> bool:
    if not _DEVICE_TERMS.search(bullet or ""): return True
    return bool(_DEVICE_TERMS.search(source or ""))

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        if re.search(r"\b(laugh|cry|kiss|tickle|massage|caress)\b", ln, re.I): continue
        if re.search(r"(?i)(thank you|pleasure taking care|discharge instructions:)", ln): continue
        # 메타/사고 문구도 추가 제거
        if re.search(_THINKING_META_RE, ln): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)
    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text
    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s or _NON_INSTR_RE.search(s): continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm); seen_bullets.add(s_norm)
                break
    if not adds: return di_text
    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

# =========================
# 8) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 9) 실행: BHC(비-RAG), DI(하이브리드 RAG) — 사고/메타 제거 적용
# =========================
N = len(sample_df)
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 생성 → 사고/메타 제거 → 불릿 강제
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(gen_model, gen_tokenizer, bhc_prompt, max_new_tokens=800)
    bhc_raw    = strip_thinking_meta(bhc_raw)  # << 사고/메타 제거
    bhc_text   = cleanup_bhc_bulleted(re.sub(r"\n{3,}", "\n\n", bhc_raw).strip())

    # ---- DI: 하이브리드 RAG
    di_exemplars, meta = retrieve_di_exemplars_hybrid(
        note_text, vs, RAG_FAISS_DIR, top_per_q=12, max_total=12
    )
    print(f"[검색요약] LLM_KW={meta.get('llm_keywords')} | 예시 {len(di_exemplars)}개")

    di_prompt = build_di_prompt_with_rag(note_text, di_exemplars)
    di_raw    = _generate_text(gen_model, gen_tokenizer, di_prompt, max_new_tokens=800)
    di_raw    = strip_thinking_meta(di_raw)  # << 1차 제거

    # ---- DI 후처리
    di_norm = normalize_bullets(ensure_bulleted(di_raw))
    di_norm = strip_thinking_meta(di_norm)   # << 2차 제거(불릿화 후 잔여)
    di_norm = reflow_wrapped_bullets(di_norm)

    uniq, seen = [], set()
    for b in di_norm.splitlines():
        s = b.strip()
        if not s or s in seen: continue
        seen.add(s); uniq.append(s)
        if len(uniq) >= 8: break
    di_norm = "\n".join(uniq)

    safe_keep = []
    for b in di_norm.splitlines():
        if not b.strip(): continue
        if not numbers_ok(b, note_text): continue
        if not device_gate(b, note_text): continue
        safe_keep.append(b)
    di_text = sanitize_di("\n".join(safe_keep))
    di_text = ensure_core_categories(di_text, note_text)
    if not di_text.strip():
        di_text = "No explicit discharge instructions found in source"

    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[warn] 4-bit 로드 실패 → 8-bit 시도: No package metadata was found for bitsandbytes


`torch_dtype` is deprecated! Use `dtype` instead!


[warn] 8-bit 로드 실패 → fp16/bf16 시도: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Qwen3-8B (generator & keyword) loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 선택 모드: 순차 슬라이스 | 시작(1-based): 40,000 | 선택 수: 3


/tmp/ipython-input-1638679019.py:185: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may b

✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[검색요약] LLM_KW=['failure to thrive', 'bile leak', 'duodenal stricture', 'gallbladder remnant', 'splenomegaly'] | 예시 1개


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
- <think>.
- </think>.
- Admission for tube feed intolerance following subtotal cholecystectomy with persistent biliary issues and failed endoscopic interventions.
- Repeat ERCP attempted but obstructed at D1/D2; right PTBD placed, followed by catheter exchange and resolution of bile leak.
- Tube feeds initiated but limited by pain and distension; adjusted to 75 cc/hr after holding for 3 days due to inadequate oral intake.
- Discharged home with service on multivitamins, acetaminophen, and vitamin D; mental status described as confused at times.
- Pending follow-up for nutritional optimization and continued monitoring of biliary function.

Discharge Instructions
- Her rehab facility has called several ti

## 그냥 콴3만 사용

In [ ]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U "bitsandbytes>=0.43.1" "transformers>=4.41.0" "accelerate>=0.30.0" "peft>=0.11.0" datasets rich
# RAG 불사용이므로 아래 패키지 설치는 필요 없습니다.
# !pip -q install -U langchain langchain-community faiss-cpu sentence-transformers rank-bm25

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000

# (단일 모델) — Qwen3-8B로 BHC/DI 생성
GEN_MODEL_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"

BASE_DATA    = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"
TEST_INPUT   = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT  = f"{BASE_DATA}/discharge_test_output.parquet"

# =========================
# 2) 모델 로더 (4bit→8bit→fp16/bf16 폴백)
# =========================
def _auto_dtype():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float32

def load_causal_safely(model_dir: str, try_4bit: bool = True):
    dtype = _auto_dtype()
    if try_4bit:
        try:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=dtype
            )
            return AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
            )
        except Exception as e:
            print(f"[warn] 4-bit 로드 실패 → 8-bit 시도: {e}")
    try:
        bnb_config = BitsAndBytesConfig(load_in_8bit=True)
        return AutoModelForCausalLM.from_pretrained(
            model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
        )
    except Exception as e:
        print(f"[warn] 8-bit 로드 실패 → fp16/bf16 시도: {e}")
    return AutoModelForCausalLM.from_pretrained(
        model_dir, torch_dtype=dtype, device_map="auto", offload_buffers=True
    )

# ===== Qwen3-8B (생성 전용) =====
gen_model = load_causal_safely(GEN_MODEL_DIR, try_4bit=True).eval()
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_DIR, trust_remote_code=True)
if gen_tokenizer.pad_token is None:
    gen_tokenizer.pad_token = gen_tokenizer.eos_token
gen_tokenizer.padding_side = "left"
gen_tokenizer.model_max_length = MAX_SEQ_LENGTH
print("✅ Qwen3-8B loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if USE_SEQUENTIAL_SLICE:
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명) — 메타/사고 금지 가드 포함
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the "Brief Hospital Course (BHC)"
for THIS admission using facts explicitly present in the INPUT note.
Do NOT include any meta-comments, chain-of-thought, reasoning steps, or apologies.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., "glomerular basement membrane (GBM)").

STYLE & CONTENT
- STYLE: Use concise bullets for scannability.
- ORDER: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at discharge → pending items.
- INCLUDE: presenting problem; salient exam/imaging/labs; pivotal decisions and treatments; trends/response; discharge status; explicit pending items.

GUARDRAILS
- Do not invent demographics/PHI.
- Do not label values as high/low/improved unless stated.

LENGTH TARGET
- TARGET: 5–8 bullets.

OUTPUT
Brief Hospital Course
- Output MUST be 5–8 bullet lines, each starting with "- ".
<content only; DO NOT output any other headings>
"""

DI_INSTR = """
You are an expert physician-writer. Compose clear, safe, patient-facing
"Discharge Instructions (DI)" for the CURRENT PATIENT based ONLY on the INPUT NOTE.
Do NOT include any meta-comments, chain-of-thought, reasoning steps, or apologies.

GOAL
- Provide safe, standard instructions suited to this patient's condition/treatments in the INPUT NOTE.

CONTENT RULES
- Do NOT invent or alter medication names/doses/frequencies/appointments not present in INPUT.
- You MAY include general guidance appropriate to the case:
  • Activity/physical limits
  • Wound/incision care
  • When to call/return to ER
  • Follow-up/monitoring
  • Diet/hydration/lifestyle

STYLE
- Write 6–8 concise bullet points.
- Each bullet begins with a directive verb (e.g., Take / Continue / Avoid / Call / Walk / Keep).
- Use simple language.

OUTPUT FORMAT
- Bulleted text only (no numbers, no subheadings).
"""

# =========================
# 5) 프롬프트/생성/후처리 유틸 (+ 사고/메타 제거)
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return gen_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, tokenizer, prompt: str, max_new_tokens: int = 800) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=1.20,
        no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def build_di_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": DI_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

# --- 사고/메타 문장 제거 정규식 & 함수 ---
_THINKING_META_RE = re.compile(
    r"(?i)\b("
    r"let me (think|start|explain)|"
    r"i (will|need to|am going to|think)|"
    r"first(ly)?|second(ly)?|to start|step[s]?:|"
    r"analysis|analysing|reasoning|thought process|chain[- ]?of[- ]?thought|rationale|"
    r"here('?s| is) (how|what)|"
    r"as an ai|as a language model|"
    r"meta(?:-)?comment|"
    r"okay,? let me|let'?s (begin|start)"
    r")\b"
)

def strip_thinking_meta(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s:
            continue
        if re.search(_THINKING_META_RE, s):
            continue
        s = re.sub(_THINKING_META_RE, "", s)
        s = re.sub(r"\s{2,}", " ", s).strip(" -:;")
        if s:
            out.append(s)
    return "\n".join(out)

def _trim(text: str, limit=800):
    return text if len(text) <= limit else text[:limit] + "\n...<trimmed>..."

_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)

def reflow_wrapped_bullets(text: str) -> str:
    out = []; cur = None
    for ln in text.splitlines():
        s = ln.rstrip()
        if not s: continue
        if s.lstrip().startswith("- "):
            if cur: out.append(cur.strip())
            cur = s.strip()
        else:
            if cur:
                joiner = " " if not cur.endswith((".", ":", ";", ",")) else " "
                cur = cur + joiner + s.strip()
            else:
                continue
    if cur: out.append(cur.strip())
    fixed = []
    for b in out:
        b2 = b
        if not re.search(r"[\.!\?]$", b2): b2 = b2 + "."
        fixed.append(b2)
    return "\n".join(fixed)

def ensure_bulleted(text: str, max_bullets: int = 8) -> str:
    if re.search(r"(?m)^\s*-\s+", text):
        text = reflow_wrapped_bullets(text)
        return "\n".join(text.splitlines()[:max_bullets])
    sents = [s.strip(" -•\t") for s in re.split(r"(?:\.\s+|\n+)", text) if s and len(s.strip()) > 0]
    bullets = [f"- {s}." if not s.endswith(".") else f"- {s}" for s in sents][:max_bullets]
    return "\n".join(bullets)

# ---- BHC 클린업 (불릿 강제 & 헤더 제거)
_BHC_SECTION_RE = re.compile(r"(?im)^\s*(TRANSITIONAL ISSUES|ADMISSION DIAGNOSES|ADMISSION TREATMENTS|RESPONSE TO CARE PLAN|Chronic Conditions|Acute Issues|New Acute Issues|New Acute Problems)\s*:.*$")
def cleanup_bhc_bulleted(text: str) -> str:
    lines = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if s.startswith("#"): continue
        if _BHC_SECTION_RE.match(s): continue
        if re.match(r"(?i)^(service|date|attending)\s*:", s): continue
        if re.match(r"(?i)^brief hospital course\b", s): continue
        lines.append(s)
    merged = "\n".join(lines)
    bul = ensure_bulleted(merged, max_bullets=8)
    return "\n".join([b for b in bul.splitlines() if b.strip()][:8])

# ---- DI 후처리/가드
def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if len(b) < 20 or len(b) > 260: continue
        if b not in seen:
            seen.add(b); final.append(b)
    return "\n".join(final)

_NUMERIC_GUARD_RE = re.compile(
    r"(?<!\w)(\d+(?:\.\d+)?)\s?(?:cc/?hr|cc|ml/?hr|ml|mg|mcg|g|units?|°f|°c|degrees|hours?|hrs?|days?|weeks?|months?|lb|lbs|pounds?|kg|kilograms?|%)\b",
    re.I
)
_DEVICE_TERMS = re.compile(r"\b(ptbd|biliary\s+(drain|catheter)|jp\s*drain|jackson-?pratt|chest\s*tube|foley|peg|nj\s*tube|ng\s*tube|wound\s+vac)\b", re.I)

def numbers_ok(bullet: str, source: str) -> bool:
    found = _NUMERIC_GUARD_RE.findall(bullet)
    if not found: return True
    src_norm = re.sub(r"\s+", " ", source).lower()
    for m in _NUMERIC_GUARD_RE.finditer(bullet):
        frag = m.group(0).lower().strip()
        if frag not in src_norm: return False
    return True

def device_gate(bullet: str, source: str) -> bool:
    if not _DEVICE_TERMS.search(bullet or ""): return True
    return bool(_DEVICE_TERMS.search(source or ""))

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        if re.search(r"\b(laugh|cry|kiss|tickle|massage|caress)\b", ln, re.I): continue
        if re.search(r"(?i)(thank you|pleasure taking care|discharge instructions:)", ln): continue
        if re.search(_THINKING_META_RE, ln): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)
    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text
    # note_text 에서 직접 추출하지 않고, 기본 템플릿 문구를 안전하게 보강
    additions = []
    if _WOUND_KW in need:
        additions.append("- Keep the incision and drain sites clean and dry; call if you notice redness, drainage, or worsening pain.")
    if _ACTIVITY_KW in need:
        additions.append("- Walk short distances several times a day; avoid heavy lifting until cleared by your clinician.")
    if _EMERG_KW in need:
        additions.append("- Call or go to the emergency room for fever over 38°C, shaking chills, severe or worsening abdominal pain, vomiting that prevents taking medicines, yellowing of the skin/eyes, or drain problems.")

    final_bullets = current_bullets + additions
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    else:
        out.append("\nDischarge Instructions")
        out.append("No explicit discharge instructions found in source.")
    return "\n".join(out).strip()

# =========================
# 6) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 7) 실행: BHC/DI — Qwen3-8B 단독 (RAG/키워드 없음)
# =========================
N = len(sample_df)
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # ---- BHC: 생성 → 사고/메타 제거 → 불릿 강제
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(gen_model, gen_tokenizer, bhc_prompt, max_new_tokens=800)
    bhc_raw    = strip_thinking_meta(bhc_raw)
    bhc_text   = cleanup_bhc_bulleted(re.sub(r"\n{3,}", "\n\n", bhc_raw).strip())

    # ---- DI: 생성 → 사고/메타 제거 → 불릿 정규화/안전가드
    di_prompt  = build_di_prompt(note_text)
    di_raw     = _generate_text(gen_model, gen_tokenizer, di_prompt, max_new_tokens=800)
    di_raw     = strip_thinking_meta(di_raw)

    di_norm = normalize_bullets(ensure_bulleted(di_raw))
    di_norm = strip_thinking_meta(di_norm)
    di_norm = reflow_wrapped_bullets(di_norm)

    # 중복/최대 8개로 제한
    uniq, seen = [], set()
    for b in di_norm.splitlines():
        s = b.strip()
        if not s or s in seen: continue
        seen.add(s); uniq.append(s)
        if len(uniq) >= 8: break
    di_norm = "\n".join(uniq)

    # 숫자/디바이스 가드
    safe_keep = []
    for b in di_norm.splitlines():
        if not b.strip(): continue
        if not numbers_ok(b, note_text): continue
        if not device_gate(b, note_text): continue
        safe_keep.append(b)
    di_text = sanitize_di("\n".join(safe_keep))
    di_text = ensure_core_categories(di_text, note_text)
    if not di_text.strip():
        di_text = "No explicit discharge instructions found in source"

    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[warn] 4-bit 로드 실패 → 8-bit 시도: No package metadata was found for bitsandbytes
[warn] 8-bit 로드 실패 → fp16/bf16 시도: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Qwen3-8B loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 선택 모드: 순차 슬라이스 | 시작(1-based): 40,000 | 선택 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
- <think>.
- </think>.
- Admission for tube feed intolerance following subtotal cholecystectomy with persistent biliary issues and failed endoscopic interventions.
- Repeat ERCP attempted but obstructed at D1/D2; right PTBD placed, followed by catheter exchange and resolution of bile leak.
- Tube feeds initiated but limited by pain and distension; adjusted to 75 cc/hr after holding for 3 days due to inadequate oral intake.
- Discharged home with service on multivitamins, acetaminophen, and vitamin D; mental status described as confused at times.
- Pending follow-up for nutritional optimization and continued monitoring of biliary function.

Discharge Instructions
- Keep the incision and drain sites clea

# Qwen3모델이 문장 추출해서 RAG검색

In [1]:
# =========================
# 설치 (필요시 1회)
# =========================
# !pip -q install -U bitsandbytes "transformers>=4.41.0" accelerate peft datasets rich
!pip -q install -U langchain langchain-community faiss-cpu sentence-transformers rank-bm25

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, numpy as np, torch
from typing import List, Tuple
from google.colab import drive
drive.mount('/content/drive')

from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
)
from peft import PeftModel

# LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

USE_SEQUENTIAL_SLICE = True
START_AT_ROW = 40000

# (생성 본체) — Qwen2-7B-Instruct 로컬 폴더 + (옵션) LoRA 어댑터
MODEL_DIR    = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR  = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.4"  # 없으면 None/빈문자열로

# (요약 전용) — Qwen3-8B (요약 5줄 생성에만 사용)
QWEN3_SUMM_DIR = "/content/drive/MyDrive/DILAB/qwen3-8b"

BASE_DATA    = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"
TEST_INPUT   = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT  = f"{BASE_DATA}/discharge_test_output.parquet"

# (RAG) DI 전용
RAG_FAISS_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/rag_files/di_index"
E5_MODEL      = "intfloat/e5-base-v2"

# =========================
# 2) 모델 로더 (bnb 자동감지: 4bit→8bit→fp16/bf16)
#    + (옵션) PEFT 어댑터 로드 & merge_unload
# =========================
def _auto_dtype():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return torch.bfloat16 if major >= 8 else torch.float16
    return torch.float32

def _has_bnb():
    try:
        import bitsandbytes as bnb  # noqa
        return True
    except Exception:
        return False

def _check_local_model_dir(path: str):
    if not os.path.isdir(path):
        raise FileNotFoundError(f"[error] 모델 폴더가 아님: {path}")
    if not os.path.exists(os.path.join(path, "config.json")):
        raise FileNotFoundError(
            f"[error] '{path}'에 config.json이 없습니다. "
            "허브에서 받은 온전한 모델 폴더인지 확인하세요 "
            "(config.json / tokenizer.json / tokenizer_config.json / model-*.safetensors 등)."
        )

def load_causal_with_optional_adapter(model_dir: str, adapter_dir: str|None, try_4bit: bool = True):
    _check_local_model_dir(model_dir)
    dtype = _auto_dtype()
    use_bnb = _has_bnb()

    base = None
    # 4bit
    if try_4bit and use_bnb:
        try:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=dtype
            )
            base = AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
            )
        except Exception as e:
            print(f"[warn] 4-bit 로드 실패 → 8-bit 시도: {e}")
    # 8bit
    if base is None and use_bnb:
        try:
            bnb_config = BitsAndBytesConfig(load_in_8bit=True)
            base = AutoModelForCausalLM.from_pretrained(
                model_dir, quantization_config=bnb_config, device_map="auto", offload_buffers=True
            )
        except Exception as e:
            print(f"[warn] 8-bit 로드 실패 → fp16/bf16 시도: {e}")
    # fp16/bf16
    if base is None:
        base = AutoModelForCausalLM.from_pretrained(
            model_dir, torch_dtype=dtype, device_map="auto", offload_buffers=True
        )

    # (옵션) LoRA 어댑터 로드
    if adapter_dir and os.path.isdir(adapter_dir):
        try:
            model = PeftModel.from_pretrained(base, adapter_dir, is_trainable=False)
            # 추론용이면 병합
            try:
                model = model.merge_and_unload()
                print("✅ PEFT adapter merged into base model.")
            except Exception as me:
                print(f"[warn] merge_and_unload 실패 → 어댑터 장착 상태로 사용: {me}")
                model = model
        except Exception as e:
            print(f"[warn] 어댑터 로드 실패(베이스만 사용): {e}")
            model = base
    else:
        model = base

    return model.eval()

def load_causal_safely(model_dir: str, try_4bit: bool = True):
    # 어댑터 없이 로딩(요약기용)
    return load_causal_with_optional_adapter(model_dir, adapter_dir=None, try_4bit=try_4bit)

# ==== (생성 본체) Qwen2-7B-Instruct ====
gen_model = load_causal_with_optional_adapter(MODEL_DIR, ADAPTER_DIR, try_4bit=True)
gen_tok   = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if gen_tok.pad_token is None:
    gen_tok.pad_token = gen_tok.eos_token
gen_tok.padding_side = "left"
gen_tok.model_max_length = MAX_SEQ_LENGTH
print("✅ Qwen2-7B-Instruct (with optional adapter) loaded")

# ==== (요약기) Qwen3-8B ====
summ_model = load_causal_safely(QWEN3_SUMM_DIR, try_4bit=True)
summ_tok   = AutoTokenizer.from_pretrained(QWEN3_SUMM_DIR, trust_remote_code=True)
if summ_tok.pad_token is None:
    summ_tok.pad_token = summ_tok.eos_token
summ_tok.padding_side = "left"
summ_tok.model_max_length = MAX_SEQ_LENGTH
print("✅ Qwen3-8B (summarizer) loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
assert "text" in test_merged_df.columns, "입력 노트 컬럼 'text'가 필요합니다."

both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if USE_SEQUENTIAL_SLICE:
    if START_AT_ROW is None or START_AT_ROW < 1:
        raise ValueError("START_AT_ROW는 1 이상의 정수여야 합니다. (1-based)")
    start_idx = START_AT_ROW - 1
    if start_idx >= len(test_merged_df):
        raise IndexError(
            f"START_AT_ROW({START_AT_ROW})가 데이터 길이({len(test_merged_df):,})를 초과합니다."
        )
    end_idx = min(start_idx + (TEST_LIMIT if TEST_LIMIT is not None else 3), len(test_merged_df))
    sample_df = test_merged_df.iloc[start_idx:end_idx].reset_index(drop=True)
    print(f"🎯 선택 모드: 순차 슬라이스 | 시작(1-based): {START_AT_ROW:,} | 선택 수: {len(sample_df):,}")
else:
    if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
        sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    else:
        sample_df = test_merged_df.copy()
    print(f"🎯 선택 모드: 무작위 샘플 | 선택 수: {len(sample_df):,}")

# =========================
# 4) 유틸 — 채팅 직렬화 & 생성
# =========================
def serialize_chat(messages: List[dict]) -> str:
    return gen_tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def serialize_chat_qwen3(messages: List[dict]) -> str:
    return summ_tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def gpt_generate(prompt: str, max_new: int = 600, do_sample: bool = False) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = gen_tok(prompt, return_tensors="pt").to(device)
    gen_ids = gen_model.generate(
      inputs.input_ids,
      max_new_tokens=max_new,
      do_sample=do_sample,
      temperature=0.7 if do_sample else None,
      top_p=0.9 if do_sample else None,
      repetition_penalty=1.12,
      no_repeat_ngram_size=5,
      eos_token_id=gen_tok.eos_token_id,
      pad_token_id=gen_tok.pad_token_id,
    )
    gen_only = [o[len(inputs.input_ids[0]):] for o in gen_ids]
    return gen_tok.decode(gen_only[0], skip_special_tokens=True).strip()

@torch.inference_mode()
def qwen3_generate(prompt: str, max_new: int = 320) -> str:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = summ_tok(prompt, return_tensors="pt").to(device)
    gen_ids = summ_model.generate(
        inputs.input_ids,
        max_new_tokens=max_new,
        do_sample=False,
        repetition_penalty=1.12,
        no_repeat_ngram_size=5,
        eos_token_id=summ_tok.eos_token_id,
        pad_token_id=summ_tok.pad_token_id,
    )
    gen_only = [o[len(inputs.input_ids[0]):] for o in gen_ids]
    return summ_tok.decode(gen_only[0], skip_special_tokens=True).strip()

# =========================
# 5) 5줄 요약 (Qwen3-8B)
# =========================
SUMM_INSTR = (
    "You are a clinical summarizer. Summarize the INPUT note in exactly 5 bullet lines.\n"
    "Rules: (1) No speculation, only facts explicitly in the note. (2) English. (3) One concise fact per line.\n"
    "Output: 5 lines, each starting with '- '. No other text."
)

def generate_summary_5lines(note_text: str, max_new_tokens: int = 320) -> str:
    messages = [
        {"role":"system", "content": SUMM_INSTR},
        {"role":"user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role":"assistant","content":""}
    ]
    prompt = serialize_chat_qwen3(messages)
    text = qwen3_generate(prompt, max_new=max_new_tokens)
    # 5줄 정규화
    lines = []
    for ln in text.splitlines():
        s = re.sub(r"^[-*•]\s*", "- ", ln.strip())
        if s.startswith("- ") and len(s) > 3:
            lines.append(s if s.endswith(".") else s + ".")
        if len(lines) >= 5:
            break
    if len(lines) < 5:
        extra = [x.strip() for x in re.split(r"\.\s+|\n+", text) if x.strip()]
        for s in extra:
            if s.lower().startswith(("brief hospital course","discharge instructions")):
                continue
            lines.append("- " + (s if s.endswith(".") else s + "."))
            if len(lines) >= 5:
                break
    return "\n".join(lines[:5]).strip()

# =========================
# 6) (RAG) DI 전용 벡터DB 로드 & 리트리버
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(RAG_FAISS_DIR, emb, allow_dangerous_deserialization=True)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

FAISS_CAND_K = 50
BM25_CAND_K  = 80
FUSE_TOPK    = 12

_BM25 = None
def _load_di_mapping(prefix: str):
    mapping_path = prefix + ".mapping.jsonl"
    if not os.path.exists(mapping_path):
        alt = os.path.join(os.path.dirname(prefix), "di_index.mapping.jsonl")
        if os.path.exists(alt): mapping_path = alt
        else: raise FileNotFoundError(f"DI mapping jsonl not found: {mapping_path}")
    texts, ids = [], []
    with open(mapping_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            txt = (obj.get("di_text","") or "").strip()
            if not txt: continue
            texts.append(txt); ids.append(obj.get("doc_id"))
    return [Document(page_content=t, metadata={"doc_id":i}) for t,i in zip(texts, ids)]

def bm25_query(bm25, query: str, k: int):
    if hasattr(bm25, "invoke"):
        docs = bm25.invoke(query)
    elif hasattr(bm25, "get_relevant_documents"):
        docs = bm25.get_relevant_documents(query)
    elif hasattr(bm25, "search"):
        docs = bm25.search(query)
    else:
        raise AttributeError("BM25 retriever has neither 'invoke' nor 'get_relevant_documents' nor 'search'.")
    return (docs or [])[:k]

def get_bm25_retriever(prefix: str):
    global _BM25
    if _BM25 is not None: return _BM25
    docs = _load_di_mapping(prefix)
    bullets = []
    for d in docs:
        for ln in d.page_content.splitlines():
            s = ln.strip()
            if len(s) < 20: continue
            if re.search(r"^(Discharge|Service|Date|Attending)\b", s, re.I): continue
            bullets.append(Document(page_content=s, metadata=d.metadata))
    _BM25 = BM25Retriever.from_documents(bullets if bullets else docs)
    _BM25.k = BM25_CAND_K
    return _BM25

from collections import defaultdict as _dd
def rrf_fuse(ranklists, k=FUSE_TOPK, c=60):
    agg = _dd(float); seen_doc = {}
    for rl in ranklists:
        for rank, (doc, _) in enumerate(rl, start=1):
            key = (doc.page_content[:200], doc.metadata.get("doc_id"))
            seen_doc[key] = doc
            agg[key] += 1.0 / (c + rank)
    fused = sorted(agg.items(), key=lambda x: x[1], reverse=True)
    return [seen_doc[k] for k,_ in fused[:k]]

# =========================
# 7) 하이브리드 RAG — "요약 5줄"을 질의로 사용
# =========================
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def retrieve_di_exemplars_from_summary(note_text: str, faiss_vs, prefix: str,
                                       top_per_q: int = 12, max_total: int = 12):
    # 1) 요약 5줄 (Qwen3-8B)
    summary5 = generate_summary_5lines(note_text)
    summary_lines = [re.sub(r"^[-*•]\s*", "", ln).strip() for ln in summary5.splitlines() if ln.strip()]
    # 2) 각 줄을 질의로 사용
    bm25 = get_bm25_retriever(prefix)
    all_docs, seen_doc = [], set()

    for q in summary_lines:
        q_e5 = f"query: {q}"
        faiss_hits = faiss_vs.similarity_search_with_score(q_e5, k=FAISS_CAND_K)
        faiss_ranked = [(d, s) for d, s in faiss_hits]
        bm25_hits = bm25_query(bm25, q, BM25_CAND_K)
        bm25_ranked = [(d, None) for d in bm25_hits]
        fused = rrf_fuse([faiss_ranked, bm25_ranked], k=top_per_q)

        for d in fused:
            t = _strip_passage_prefix(d.page_content.strip())
            if 60 <= len(t) <= 1200 and not _NON_INSTR_RE.search(t):
                key = (t[:200], d.metadata.get("doc_id"))
                if key not in seen_doc:
                    seen_doc.add(key); all_docs.append(t)
            if len(all_docs) >= max_total: break
        if len(all_docs) >= max_total: break

    # 3) 예시 문장 정리(불릿만 추출)
    bullets = []
    for t in all_docs:
        for ln in t.splitlines():
            s = ln.strip()
            if not s or _NON_INSTR_RE.search(s): continue
            if s.startswith(("-", "*")) or re.search(r"^(Call|Avoid|Continue|Take|Keep|Walk|Shower|Do not|Don't|Monitor|Check)\b", s, re.I):
                s = re.sub(r"^[-*•]\s*", "- ", s)
                bullets.append(s)
    uniq, bag = [], set()
    for b in bullets:
        if b not in bag:
            bag.add(b); uniq.append(b)
        if len(uniq) >= max_total: break
    exemplars = []
    if len(uniq) >= 6:
        exemplars.append("\n".join(uniq[:12]))
    else:
        exemplars.extend(all_docs[:6])

    meta = {"summary5": summary5}
    return exemplars, meta

# =========================
# 8) Qwen2-Instruct용 프롬프트 & 생성
#  (BHC 지시문: 사용자 스펙 반영, DI는 기존 유지)
# =========================
def serialize_chat(messages: List[dict]) -> str:
    return gen_tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# ====== BHC 전용 인스트럭션 (사용자 제공 스펙) ======
INSTRUCTION_BHC = """
You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Maintain chronologic clarity (e.g., “On POD #1...”, “By discharge...” when timing is available).
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → daily or staged progression if described → response/status at discharge → pending studies/follow-up items.

OUTPUT FORMAT
Brief Hospital Course
- Summarize the course of THIS admission only.
- Include: presenting problem; salient exam/imaging/labs (with values only if meaningful); pivotal decisions and treatments (med changes with stated reasons); postoperative or inpatient progression by day/stage if mentioned; clinical response/trends; discharge status; and explicit pending items for follow-up.
- Explicitly mention removal or initiation of tubes, lines, diet advancement, or ambulation if these are provided in input.

GUARDRAILS
- No new clinicians, appointments, or tests beyond the input.
- Do not invent demographics/PHI (age, phone, MRN, addresses).
- Do not label values as high/low/improved unless the input does so.
- Do NOT repeat any disease/problem header more than once. If the same problem appears multiple times in the input, summarize it once and merge details.
- Each header beginning with “#” must be unique; combine duplicate/near-duplicate sections.

FINALIZATION (silent self-check)
- Output ONLY the 'Brief Hospital Course' bullets in this step (no DI).
- Every sentence is supported by the input; all numbers/units/doses match verbatim when cited.
- BHC shows time-based/staged progression when available and a clear discharge status.
"""

DI_GEN_RULES = """
You are an expert physician-writer. Compose patient-facing "Discharge Instructions (DI)" for THIS patient,
using ONLY INPUT note facts. Use RAG exemplars for style/structure only; do not invent doses/clinicians/appointments.
- 6–8 concise bullets; each starts with a directive verb (Take/Continue/Avoid/Call/Walk/Keep...).
- Include only actionable items explicitly present, plus safe general guidance (activity/wound care/when-to-call) if appropriate.
- No medication list restatement unless given as instructions.
- Avoid meta-comments or apologies.
Output: bullets only (no numbering, no extra headers).
"""

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role":"system","content": INSTRUCTION_BHC + "\n\nFor this step, output ONLY 5–8 bullet lines under 'Brief Hospital Course'."},
        {"role":"user","content":f"INPUT NOTE:\n{note_text.strip()}"},
        {"role":"assistant","content":""}
    ]
    return serialize_chat(messages)

def build_di_prompt(note_text: str, di_exemplars: List[str], summary5: str) -> str:
    exemplars_block = "\n\n".join([f"### DI exemplar #{i+1}\n{ex}" for i,ex in enumerate(di_exemplars)]) if di_exemplars else "(none)"
    messages = [
        {"role":"system","content":DI_GEN_RULES},
        {"role":"user","content":(
            f"INPUT NOTE:\n{note_text.strip()}\n\n"
            f"5-LINE SUMMARY (from Qwen3-8B):\n{summary5}\n\n"
            f"RETRIEVED DI EXEMPLARS (style only):\n{exemplars_block}\n"
        )},
        {"role":"assistant","content":""}
    ]
    return serialize_chat(messages)

# --- BHC 후처리: DI 스타일 문구 제거(안전장치) ---
_DI_DIRECTIVE_RE = re.compile(
    r"^(call|avoid|do not|don't|continue|take|keep|walk|shower|monitor|check|return|seek|go to)\b",
    re.I
)
_DI_EMERG_RE = re.compile(
    r"(fever\s*[>≥]\s*101|shortness of breath|chest pain|black|tarry\s*stool|vomit|dehydration|worsen\w*\s+pain)",
    re.I
)
def cleanup_bhc_lines(text: str) -> str:
    # 불릿만 추출
    raw = []
    for ln in text.splitlines():
        s = re.sub(r"^[-*•]\s*", "- ", ln.strip())
        if s.startswith("- ") and len(s) > 3:
            raw.append(s)
    # DI 스타일 제거
    keep = []
    for b in raw:
        body = b[2:].strip()
        if _DI_DIRECTIVE_RE.search(body):  # 지시 동사 시작
            continue
        if _DI_EMERG_RE.search(body):     # 응급 경고 패턴
            continue
        keep.append(b if b.endswith(".") else b + ".")
        if len(keep) >= 8:
            break
    # 최소 5줄 확보 실패 시 원문에서 비지시형 문장으로 보충
    if len(keep) < 5:
        for b in raw:
            if b in keep: continue
            if not _DI_DIRECTIVE_RE.search(b[2:]) and not _DI_EMERG_RE.search(b[2:]):
                keep.append(b if b.endswith(".") else b + ".")
            if len(keep) >= 5: break
    return "\n".join(keep[:8])

# =========================
# 9) 정답(GT) 준비
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 10) 실행: (1) 5줄 요약(Qwen3) → (2) 요약 기반 RAG → (3) Qwen2로 BHC/DI 생성
# =========================
for i in range(len(sample_df)):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{len(sample_df)} ---------------")

    # (1) 5줄 요약 — Qwen3-8B
    summary5 = generate_summary_5lines(note_text)
    print("[요약(5줄, Qwen3-8B)]:")
    print(summary5)

    # (2) 요약을 질의로 하이브리드 RAG 검색
    di_exemplars, meta = retrieve_di_exemplars_from_summary(
        note_text, vs, RAG_FAISS_DIR, top_per_q=12, max_total=12
    )
    print(f"[RAG] 예시 개수: {len(di_exemplars)}")

    # (3-a) BHC 생성 — Qwen2-7B-Instruct + 새로운 BHC 지시문
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = gpt_generate(bhc_prompt, max_new=520, do_sample=False)
    bhc_text   = cleanup_bhc_lines(bhc_raw)  # 안전장치: DI문구 제거
    final_bhc  = "Brief Hospital Course\n" + (bhc_text if bhc_text.strip() else "- (no content).")

    # (3-b) DI 생성 — Qwen2-7B-Instruct (RAG 예시 + 요약5 제공)
    di_prompt  = build_di_prompt(note_text, di_exemplars, summary5)
    di_text    = gpt_generate(di_prompt, max_new=520, do_sample=False)

    # 출력 정리(불릿 정규화; DI만)
    def _ensure_bullets(t: str, header: str) -> str:
        lines = []
        for ln in t.splitlines():
            s = re.sub(r"^[-*•]\s*", "- ", ln.strip())
            if s.startswith("- ") and len(s) > 3:
                lines.append(s if s.endswith(".") else s + ".")
        if not lines:
            sents = [x.strip() for x in re.split(r"(?:\.\s+|\n+)", t) if x.strip()]
            lines = [("- " + s + ("" if s.endswith(".") else ".")) for s in sents][:8]
        return header + "\n" + "\n".join(lines[:8]).strip()

    final_di  = _ensure_bullets(di_text, "Discharge Instructions")
    final_out = f"{final_bhc}\n\n{final_di}"

    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_out)

    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ PEFT adapter merged into base model.
✅ Qwen2-7B-Instruct (with optional adapter) loaded


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Qwen3-8B (summarizer) loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 선택 모드: 순차 슬라이스 | 시작(1-based): 40,000 | 선택 수: 3


/tmp/ipython-input-3077744322.py:266: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may b

✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
[요약(5줄, Qwen3-8B)]:
- The patient is a female with a history of subtotal cholecystectomcy and a bile leak following a failed ERCP due to duodenol stricture.
- She underwent an over-the-wire cholanjiogram and exchange of a percutaneous trans-hepatic biliary drainage catheter.
- Tube feeds were initiated but discontinued due to pain and distension; she was later transferred for evaluation of emesis.
- Recent labs show elevated calcium, phosphorus, and magnesium levels, along with mild anemia and abnormal liver function tests.
- Discharged home with service, diagnosed with failure to thrive, and prescribed multivitamins, acetaminophen, and vitamin D.


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[RAG] 예시 개수: 6


-------------------FINAL MODEL OUTPUT-------------------
Brief Hospital Course
- Please continue to monitor her tube feeds and ensure that she.
- Please continue to monitor her tube feeds and ensure that she.

Discharge Instructions
- Take your medications as prescribed.
- Continue to take your tube feeds as directed.
- Walk multiple times per day.
- Call your doctor if you experience increased abdominal pain,.
- fever > 101.5°F, decreased urine output, or any concerning.
- symptoms.
- ___.


-------------------output (GT)------------------------
BHC (GT)
The patient was transferred from rehabilitation and admitted to 
the HPB Surgical Service for evaluation secondary to tube feed 
intolerance. She underwent abdominal CT scan, which demonstrated 
phlegmonous changes in the gallbladder fossa which may represent 
remnant gallbladder or new fluid collection, and slight interval 
increase in intrahepatic biliary dilatation. Patient was 
afebrile on admission with normal WB